# Servidor con generador de voz sintética y traducción en ambas direcciones usando zephyr model.

Ten en cuenta activar CORS para que no de error:
%pip install flask-cors

In [29]:
%%writefile server_conversacional.py
from flask import Flask, request, jsonify
from flask_cors import CORS
from transformers import MarianMTModel, MarianTokenizer
from threading import Lock
import torch

import time
# Asegúrate de importar tu modelo Whisper correctamente
# from tu_paquete import modelWhisper

import whisper
modelWhisper = whisper.load_model('medium')


model_name = 'Helsinki-NLP/opus-mt-es-en'  # Modelo para traducir de español a inglés
modelo_traductor = MarianMTModel.from_pretrained(model_name)
tokenizer = MarianTokenizer.from_pretrained(model_name)


def translate_text_to_english(text):
    print("Traduciendo texto:", text)
    tokens = tokenizer(text, return_tensors='pt', padding=True)
    translated = modelo_traductor.generate(**tokens)
    decoded = []
    for t in translated:
        decoded.append(tokenizer.decode(t, skip_special_tokens=True))

    return decoded[0]


model_name = 'Helsinki-NLP/opus-mt-tc-big-en-es'  # Modelo para traducir de inglés a español
model_big_en_es = MarianMTModel.from_pretrained(model_name)
tokenizer_big_en_es = MarianTokenizer.from_pretrained(model_name)

def translate_text_to_spanish(text):
      tokenizer = tokenizer_big_en_es
      model = model_big_en_es

      tokens = tokenizer(text, return_tensors='pt', padding=True)
      translated = model.generate(**tokens)
      # decoded = tokenizer.decode(translation[0], skip_special_tokens=True)
      decoded = []
      for t in translated:
          # decoded = tokenizer.decode(t, skip_special_tokens=True)
          decoded.append(tokenizer.decode(t, skip_special_tokens=True))
      
      return decoded[0]


modelo = "zypher"

if modelo == "mistral":
    from modelo_mistral_base import generate_long_chat, load_model
elif modelo == "zypher":
    from modelo_Zypher_beta import generate_long_chat, load_model
else:
    print("modelo no encontrado")
    exit()

ai = "assistant"
user = "user"

contexto = """

"""

system_prompt = """
You are a kind and helpful assistan bot. You are here to help the user to find the best answer to his question.
"""

saludo = "Hello, I am ready to receive and process your input."

idioma = "en"

import sys

# Verifica si el comando tenía flag -s o --short
if "-s" in sys.argv or "--short" in sys.argv:
    short_answer = True

# Si encuentra el flag -es cambia el idioma a español
if "-es" in sys.argv:
    idioma = "es"

# Filtra los argumentos para eliminar los flags
args = [arg for arg in sys.argv[1:] if arg not in ["-s", "--short", "-es"]]

# Asigna los valores a system_prompt y saludo basándose en los argumentos restantes
if len(args) > 0:
    system_prompt = args[0]
if len(args) > 1:
    saludo = args[1]

if modelo == "mistral":
    historico = f"<|im_start|>system\n{system_prompt}<|im_end|>\n<|im_start|>assistant\n{saludo}<|im_end|>\n"
elif modelo == "zypher":
    historico = f"<|system|>{system_prompt}</s>\n<|assistant|>\n{saludo}</s>\n"


# load model
load_model(user=user, ai=ai)

print(f"{ai}:", saludo)

# Crea un bloqueo para proteger el código contra la concurrencia a la hora de transcribir
transcribe_lock = Lock()

# Crea un bloqueo para proteger el código contra la concurrencia a la hora de traducir
translate_lock_es = Lock()

translate_lock_en = Lock()

# Crea un bloqueo para proteger el código contra la concurrencia a la hora de generar texto
generate_lock = Lock()




app = Flask(__name__)
# app.config['MAX_CONTENT_LENGTH'] = 30 * 1024 * 1024  # 30 MB

CORS(app)


@app.route('/inicio', methods=['POST'])
def print_strings():
    global modelo
    global historico

    # Obtiene los datos del cuerpo de la solicitud
    data = request.json
    
    # Extrae los strings del objeto JSON
    system_prompt = data.get('system_prompt')
    saludo = data.get('saludo')
    
    # Imprime los strings en el log del servidor
    print("INICIALIZANDO CONVERSACIÓN")
    print(f"system: {system_prompt}, saludo: {saludo}")
    
    if modelo == "mistral":
        historico = f"<|im_start|>system\n{system_prompt}<|im_end|>\n<|im_start|>assistant\n{saludo}<|im_end|>\n"
    elif modelo == "zypher":
        historico = f"<|system|>{system_prompt}</s>\n<|assistant|>\n{saludo}</s>\n"


    # Retorna una respuesta para indicar que se recibieron y procesaron los datos
    return jsonify({"message": "Strings received and conversation initialized."}), 200



@app.route('/transcribe', methods=['POST'])
def transcribe_audio():
    print("Transcribiendo audio...")
    global historico
    global user
    global ai
    # global iteracion

    traduccion = ""

    # Comprueba si el archivo fue enviado
    if 'file' not in request.files:
        return jsonify(error="No file part"), 400


    file = request.files['file']

    # Comprueba si el usuario no seleccionó un archivo
    if file.filename == '':
        return jsonify(error="No selected file"), 400

    # Genera un nombre de archivo único utilizando una marca de tiempo
    timestamp = int(time.time() * 1000)  # Marca de tiempo en milisegundos

    
    mp3_filepath = f"received_audio_{timestamp}.mp3"
    file.save(mp3_filepath)

    # Transcribe el archivo MP3 (Asegúrate de tener el modelo cargado correctamente)
    # Transcribe el archivo MP3 dentro de una sección crítica protegida por un bloqueo
    with transcribe_lock:
        # transcripcion = modelWhisper.transcribe(mp3_filepath, fp16=False)
        # transcipción lenguaje inglés
        transcripcion = modelWhisper.transcribe(mp3_filepath, fp16=False, language=idioma)
        transcripcion = transcripcion["text"]
    print("transcripción:", transcripcion)

    # si el idioma es español, traduce la transcripción al inglés
    if idioma == "es":
        with translate_lock_en:
            traduccion = translate_text_to_english(transcripcion)
        print("traducción:", traduccion)
        entrada = traduccion
    else:
        entrada = transcripcion
    # prompt = f"{historico}\n{user}:{entrada}\n{ai}:"
    # print("prompt:", prompt)

    with generate_lock:
        historico, output = generate_long_chat(historico, ai, user, input_text=entrada, max_additional_tokens=2048, short_answer=short_answer, streaming=False, printing=False)
        print("output:", output)
        print("historico:", historico)
      
    # si el idioma es español, traduce la respuesta al español
    if idioma == "es":
        with translate_lock_es:
            traduccion_res = translate_text_to_spanish(output)
        print("traducción:", traduccion_res)
        respuesta_trad = traduccion_res



    return jsonify(entrada=transcripcion, entrada_traducida=traduccion, respuesta=output, respuesta_traducida=respuesta_trad)



@app.route('/texto', methods=['POST'])
def process_text():
    global historico
    global user
    global ai

    # Recibe el texto directamente del cuerpo de la solicitud
    data = request.json
    if not data or 'texto' not in data:
        return jsonify(error="No se proporcionó texto"), 400

    texto = data['texto']
    
    # Utiliza la variable 'idioma' declarada globalmente
    global idioma

    # Si el idioma es español, traduce la transcripción al inglés
    traduccion = ""
    if idioma == "es":
        with translate_lock_es:
            traduccion = translate_text_to_english(texto)
        entrada = traduccion
    else:
        entrada = texto

    # Generación de respuesta basada en el texto proporcionado
    with generate_lock:
        historico, output = generate_long_chat(historico, ai, user, input_text=entrada, max_additional_tokens=2048, short_answer=short_answer, streaming=False, printing=False)

    # si el idioma es español, traduce la respuesta al español
    if idioma == "es":
        with translate_lock_es:
            traduccion_res = translate_text_to_spanish(output)
        print("traducción:", traduccion_res)
        respuesta_trad = traduccion_res



    return jsonify(entrada=texto, entrada_traducida=traduccion, respuesta=output, respuesta_traducida=respuesta_trad)



# PREPARAMOS INSTANCIAS DE OpenVoice

if idioma == "en":
    import os
    openvoice_dir = os.path.abspath("OpenVoice")
    import sys
    sys.path.append(openvoice_dir)

    import torch
    import se_extractor
    from api import BaseSpeakerTTS, ToneColorConverter

    open_voice_dir = '/content/OpenVoice'
    output_dir = f"{open_voice_dir}/output"
    save_path = f'{output_dir}/output_en_default.wav'

    ckpt_base = f"{open_voice_dir}/checkpoints/base_speakers/EN"
    ckpt_converter = f"{open_voice_dir}/checkpoints/converter"
    device="cuda:0" if torch.cuda.is_available() else "cpu"

    base_speaker_tts = BaseSpeakerTTS(f'{ckpt_base}/config.json', device=device)
    base_speaker_tts.load_ckpt(f'{ckpt_base}/checkpoint.pth')

    tone_color_converter = ToneColorConverter(f'{ckpt_converter}/config.json', device=device)
    tone_color_converter.load_ckpt(f'{ckpt_converter}/checkpoint.pth')

    os.makedirs(output_dir, exist_ok=True)


    source_se = torch.load(f'{ckpt_base}/en_style_se.pth').to(device)
    # source_se = torch.load(f'{ckpt_base}/en_default_se.pth').to(device)

    reference_speaker = f"{open_voice_dir}/resources/demo_speaker0.mp3"
    # reference_speaker = "resources/javier.mp3"
    target_se, audio_name = se_extractor.get_se(reference_speaker, tone_color_converter, target_dir='processed', vad=True)


    save_path = f'{output_dir}/output_whispering.wav'



elif idioma == "es":
    # El modelo no entiende de números aritméticos. Esta función los convierte a palabras.
    import re
    from num2words import num2words

    def number_to_words(num_str):
        try:
            num = int(num_str)
            return num2words(num, lang='es')
        except ValueError:
            return "Por favor, introduzca un número válido."

    def process_numbers_in_line(line):
        def replace_with_words(match):
            return number_to_words(match.group())
        
        return re.sub(r'\b\d+\b', replace_with_words, line)

    # Ejemplo de uso
    line = "Tengo 3 manzanas y 15 naranjas, sumando un total de 18 frutas."
    new_line = process_numbers_in_line(line)
    print(new_line)  
    # Salida: "Tengo tres manzanas y quince naranjas, sumando un total de dieciocho frutas."


    # Diccionario con las traducciones

    def process_abrev(line):
        translations = {
        'Dr': 'doctor',
        'Sr': 'señor',
        'Sra': 'señora',
        # Añade más traducciones aquí
    }
        for abbr, full in translations.items():
            line = line.replace(f'{abbr}.', full)
            line = line.replace(f'{abbr} ', f'{full} ')
        return line

    def otras_traducciones(line):

        translations = {
        '-': ',',
        '—': ',', 
        '%': ' por ciento '
        # Añade más traducciones aquí
        }
        
        for old, new in translations.items():
            line = line.replace(old, new)
        return line        


    def preprocesado_al_modelo(line):
        line_with_numbers = process_numbers_in_line(line)
        line_with_both = process_abrev(line_with_numbers)
        line_with_all = otras_traducciones(line_with_both)
        return line_with_all

    import os
    os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'
    print(os.environ['TF_ENABLE_ONEDNN_OPTS'])


    from pydub import AudioSegment, silence

    def quitar_silencios(input_filepath, output_filepath, min_silence_len=1500, new_silence_len=750, silence_thresh=-60):
        """
        Elimina silencios largos de un archivo de audio.

        Parámetros:
        - input_filepath: ruta al archivo de audio de entrada (MP3).
        - output_filepath: ruta al archivo de audio de salida (MP3).
        - min_silence_len: duración mínima del silencio a eliminar (en milisegundos).
        - new_silence_len: duración de los nuevos segmentos de silencio (en milisegundos).
        - silence_thresh: umbral de silencio (en dB).
        """
        
        # Cargar el archivo de audio
        audio_segment = AudioSegment.from_wav(input_filepath)
        
        # Encuentra los segmentos de audio separados por silencios
        segments = silence.split_on_silence(audio_segment, min_silence_len=min_silence_len, silence_thresh=silence_thresh)

        # Crear un nuevo segmento de audio con silencios ajustados
        new_audio_segment = AudioSegment.empty()
        silence_chunk = AudioSegment.silent(duration=new_silence_len)  # Chunk de silencio de la nueva duración

        # Añade cada segmento de audio al nuevo audio, intercalando con los nuevos segmentos de silencio
        for segment in segments:
            new_audio_segment += segment + silence_chunk

        # Removemos el último chunk de silencio añadido
        new_audio_segment = new_audio_segment[:-new_silence_len]

        # Guarda el nuevo archivo de audio
        new_audio_segment.export(output_filepath, format="wav")


    # Cargamos el modelo generador fairseq
    from fairseq.checkpoint_utils import load_model_ensemble_and_task_from_hf_hub
    from fairseq.models.text_to_speech.hub_interface import TTSHubInterface
    import IPython.display as ipd
    import torch  # Importamos PyTorch para poder usar la función `to()`

    # Función para mover recursivamente todos los tensores en una estructura anidada a un dispositivo
    def move_to_device(obj, device):
        if isinstance(obj, torch.Tensor):
            return obj.to(device)
        elif isinstance(obj, dict):
            return {key: move_to_device(value, device) for key, value in obj.items()}
        elif isinstance(obj, list):
            return [move_to_device(item, device) for item in obj]
        else:
            return obj
    

    # Cargamos el modelo y la configuración desde el modelo preentrenado de Hugging Face
    models, cfg, task = load_model_ensemble_and_task_from_hf_hub(
        "facebook/tts_transformer-es-css10",
        arg_overrides={"vocoder": "hifigan", "fp16": False}
    )
    model = models[0]

    # Movemos el modelo al dispositivo GPU
    model = model.to('cuda:0')

    # Actualizamos la configuración con los datos del task
    TTSHubInterface.update_cfg_with_data_cfg(cfg, task.data_cfg)

    # Creamos el generador
    generator = task.build_generator([model], cfg)

    
    import torchaudio

    import re

    def dividir_texto_con_minimo_palabras(texto, min_palabras=8):
        partes = re.split(r'([.,;:?!])', texto)
        partes_filtradas = [parte.strip() for parte in partes if parte.strip()]
        partes_combinadas = []
        parte_actual = ''

        for parte in partes_filtradas:
            if parte in '.,;:?!':
                parte_actual += parte
                if len(parte_actual.split()) >= min_palabras:
                    partes_combinadas.append(parte_actual)
                    parte_actual = ''
                else:
                    parte_actual += ' '
            else:
                parte_actual += parte + ' '

        if len(parte_actual.strip()) > 10:
            partes_combinadas.append(parte_actual.strip())

        return partes_combinadas

    def combinar_audios(audios_temporales):
        audio_combinado = "audio_combinado.wav"
        # Cargar el primer archivo de audio para inicializar la concatenación
        wav_total, rate = torchaudio.load(audios_temporales[0])

        # Iterar sobre los archivos restantes y concatenarlos
        for archivo in audios_temporales[1:]:
            wav, _ = torchaudio.load(archivo)
            wav_total = torch.cat((wav_total, wav), 1)

        # Guardar el audio combinado en un archivo final
        torchaudio.save(audio_combinado, wav_total, rate)

        return audio_combinado

    def voz_sintetica_spanish(text):
        text = preprocesado_al_modelo(text)

        lista_dividida = dividir_texto_con_minimo_palabras(text)

        audios_temporales = []
        
        for parte in lista_dividida:
            # Preparamos los datos de entrada para el modelo
            sample = TTSHubInterface.get_model_input(task, parte)

            # Movemos los datos al dispositivo GPU
            sample = move_to_device(sample, 'cuda:0')

            # Realizamos la predicción
            wav, rate = TTSHubInterface.get_prediction(task, model, generator, sample)

            if len(wav.shape) == 1:
                wav = wav.unsqueeze(0)
                
            # temp_file_name = "Temporal.wav"
            temp_file_name = f"temporal_{parte[:10]}.wav"
            torchaudio.save(temp_file_name, wav.to('cpu'), rate)
            audios_temporales.append(temp_file_name)

            combinado = combinar_audios(audios_temporales)
            sin_silencios = "sin_silencios.wav"
            # quitamos silencios
            quitar_silencios(combinado, sin_silencios, min_silence_len=1500, new_silence_len=750, silence_thresh=-60)


        with open(sin_silencios, "rb") as audio_file:
            audio_base64 = base64.b64encode(audio_file.read()).decode('utf-8')

        return audio_base64



import base64
@app.route('/audio', methods=['POST'])
def generate_audio():
    texto = request.json.get('texto', '')

    if not texto:
        return jsonify(error="No se proporcionó texto"), 400

    if idioma == "en":
        audio_base64 = voz_sintetica_english(texto)
        return jsonify(audio_base64=audio_base64)
    elif idioma == "es":
        audio_base64 = voz_sintetica_spanish(texto)
        return jsonify(audio_base64=audio_base64)

def voz_sintetica_english(texto):
    src_path = 'tmp.wav'

    base_speaker_tts.tts(texto, src_path, speaker='default', language='English', speed=0.9)
  # Run the tone color converter
    encode_message = "@MyShell"
    tone_color_converter.convert(
        audio_src_path=src_path,
        src_se=source_se,
        tgt_se=target_se,
        output_path=save_path,
        message=encode_message)

    with open(save_path, "rb") as audio_file:
        audio_base64 = base64.b64encode(audio_file.read()).decode('utf-8')

    return audio_base64



if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5500, threaded=True)


Overwriting server_conversacional.py


SERVIDOR SIN TRADUCCIONES:

In [2]:
%%writefile server_conversacional.py
from flask import Flask, request, jsonify, send_file
from flask_cors import CORS
from threading import Lock
import threading
import os
import torch
from pydub import AudioSegment
import pandas as pd
import random

import time


import whisper
modelWhisper = whisper.load_model('medium')


modelo = "zypher"

if modelo == "mistral":
    from modelo_mistral_base import generate_long_chat, load_model
elif modelo == "zypher":
    from modelo_Zypher_beta import generate_long_chat, load_model
else:
    print("modelo no encontrado")
    exit()

ai = "assistant"
user = "user"

contexto = """

"""

system_prompt = """
You are a kind and helpful assistan bot. You are here to help the user to find the best answer to his question.
"""

saludo = "Hello, I am ready to receive and process your input."

idioma = "en"

import sys

# Verifica si el comando tenía flag -s o --short
if "-s" in sys.argv or "--short" in sys.argv:
    short_answer = True

# Si encuentra el flag -es cambia el idioma a español
if "-es" in sys.argv:
    idioma = "es"

# Filtra los argumentos para eliminar los flags
args = [arg for arg in sys.argv[1:] if arg not in ["-s", "--short", "-es"]]

# Asigna los valores a system_prompt y saludo basándose en los argumentos restantes
if len(args) > 0:
    system_prompt = args[0]
if len(args) > 1:
    saludo = args[1]

if modelo == "mistral":
    historico = f"<|im_start|>system\n{system_prompt}<|im_end|>\n<|im_start|>assistant\n{saludo}<|im_end|>\n"
elif modelo == "zypher":
    historico = f"<|system|>{system_prompt}</s>\n<|assistant|>\n{saludo}</s>\n"


# load model
load_model(user=user, ai=ai)

print(f"{ai}:", saludo)

# Crea un bloqueo para proteger el código contra la concurrencia a la hora de transcribir
transcribe_lock = Lock()


# Crea un bloqueo para proteger el código contra la concurrencia a la hora de generar texto
generate_lock = Lock()




app = Flask(__name__)
# app.config['MAX_CONTENT_LENGTH'] = 30 * 1024 * 1024  # 30 MB

CORS(app)


@app.route('/inicio', methods=['POST'])
def print_strings():
    global modelo
    global historico

    # Lee el archivo CSV y selecciona un personaje de ficción al azar
    def elegir_personaje_aleatorio():
        df = pd.read_csv('Personajes_ficcion.csv')
        return random.choice(df.iloc[:, 0].tolist())

    # Obtiene los datos del cuerpo de la solicitud
    data = request.json

    # Extrae los strings del objeto JSON
    system_prompt = data.get('system_prompt')
    saludo = data.get('saludo')

    # Preprocesamiento para reemplazar "#personaje" con un personaje aleatorio
    if "#personaje" in system_prompt:
        personaje_aleatorio = elegir_personaje_aleatorio()
        system_prompt = system_prompt.replace("#personaje", personaje_aleatorio)

    # Preprocesamiento para reemplazar "#personaje" con un personaje aleatorio en el saludo
    if "#personaje" in saludo:
        saludo = saludo.replace("#personaje", personaje_aleatorio)

    # Imprime los strings en el log del servidor
    print("INICIALIZANDO CONVERSACIÓN")
    conversation_file = 'conversacion.mp3'
    # si existe el archivo de conversación, lo elimina
    if os.path.exists(conversation_file):
        os.remove(conversation_file)
        
    print(f"system: {system_prompt}, saludo: {saludo}")

    if modelo == "mistral":
        historico = f"system\n{system_prompt}\nassistant\n{saludo}\n"
    elif modelo == "zypher":
        historico = f"{system_prompt}</s>\n\n{saludo}</s>\n"

    # Retorna una respuesta para indicar que se recibieron y procesaron los datos
    return jsonify({"message": saludo}), 200



@app.route('/all_conversation', methods=['GET'])
def all_conversation():
    # Asegúrate de que el path al archivo sea correcto para tu estructura de proyecto
    filepath = 'conversacion.mp3'
    if not os.path.exists(filepath):
        return jsonify(error="Archivo de conversación no encontrado"), 404

    # Leer el archivo y convertirlo a base64
    with open(filepath, 'rb') as audio_file:
        audio_base64 = base64.b64encode(audio_file.read()).decode('utf-8')

    return jsonify(audio_base64=audio_base64)



import subprocess

def convert_ogg_to_mp3(source_ogg_path, target_mp3_path):
    """
    Utiliza ffmpeg para convertir un archivo .ogg a .mp3.
    """
    command = ['ffmpeg', '-y' ,'-i', source_ogg_path, target_mp3_path]
    process = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

    # Si el comando falla, imprime la salida de error
    if process.returncode != 0:
        print(f"Error al convertir {source_ogg_path} a {target_mp3_path}")
        print("Salida de error de ffmpeg:")
        print(process.stderr.decode())




def convert_wav_to_mp3(source_wav_path, target_mp3_path):
    command = ['ffmpeg', '-i', source_wav_path, target_mp3_path]
    subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE)


def add_audio_to_conversation_async(source_path, convert_to_mp3=False):
    def task():
        if convert_to_mp3:
            # Convertir de WAV a MP3 si es necesario
            temp_mp3_path = source_path.replace('.wav', '.mp3')
            convert_wav_to_mp3(source_path, temp_mp3_path)
            final_path = temp_mp3_path
        else:
            final_path = source_path

        # Añadir al archivo de conversación
        sound = AudioSegment.from_file(final_path)
        conversation_file = 'conversacion.mp3'
        if os.path.exists(conversation_file):
            conversation_audio = AudioSegment.from_mp3(conversation_file)
            combined_audio = conversation_audio + sound
        else:
            combined_audio = sound
        combined_audio.export(conversation_file, format='mp3')
        
        # Limpiar archivos temporales
        os.remove(source_path)
        if convert_to_mp3:
            os.remove(temp_mp3_path)

    thread = threading.Thread(target=task)
    thread.start()



@app.route('/transcribe', methods=['POST'])
def transcribe_audio():
    print("Transcribiendo audio...")
    global historico
    global user
    global ai
    # global iteracion



    # Comprueba si el archivo fue enviado
    if 'file' not in request.files:
        return jsonify(error="No file part"), 400


    file = request.files['file']

    # Comprueba si el usuario no seleccionó un archivo
    if file.filename == '':
        return jsonify(error="No selected file"), 400

    # Genera un nombre de archivo único utilizando una marca de tiempo
    timestamp = int(time.time() * 1000)  # Marca de tiempo en milisegundos


    ogg_filepath = f"received_audio_{timestamp}.ogg"
    file.save(ogg_filepath)

    start_transcribe_time = time.time()  # Inicio de la transcripción

    # Transcribe el archivo MP3 dentro de una sección crítica protegida por un bloqueo
    with transcribe_lock:
        # transcripcion = modelWhisper.transcribe(mp3_filepath, fp16=False)
        # transcipción lenguaje inglés
        transcripcion = modelWhisper.transcribe(ogg_filepath, fp16=False, language=idioma)
        transcripcion = transcripcion["text"]
    
    end_transcribe_time = time.time()  # Fin de la transcripción
    transcribe_duration = end_transcribe_time - start_transcribe_time
    print(f"Transcripción completada en {transcribe_duration} segundos")        
    print("transcripción:", transcripcion)

    # si el idioma es español, traduce la transcripción al inglés

    entrada = transcripcion
    # prompt = f"{historico}\n{user}:{entrada}\n{ai}:"
    # print("prompt:", prompt)

    start_generation_time = time.time()  # Inicio de la generación
    with generate_lock:
        historico, output = generate_long_chat(historico, ai, user, input_text=entrada, max_additional_tokens=2048, short_answer=short_answer, streaming=False, printing=False)

    end_generation_time = time.time()  # Fin de la generación
    generation_duration = end_generation_time - start_generation_time
    print(f"Generación completada en {generation_duration} segundos")
    
    print("output:", output)
    print("historico:", historico)


    # Inicia el proceso de adición del audio .ogg en segundo plano, considerando su conversión a .mp3
    add_audio_to_conversation_async(ogg_filepath)

    return jsonify(entrada=transcripcion, entrada_traducida="", respuesta=output, respuesta_traducida="")



@app.route('/texto', methods=['POST'])
def process_text():
    global historico
    global user
    global ai

    # Recibe el texto directamente del cuerpo de la solicitud
    data = request.json
    if not data or 'texto' not in data:
        return jsonify(error="No se proporcionó texto"), 400

    texto = data['texto']

    # Utiliza la variable 'idioma' declarada globalmente
    global idioma

    entrada = texto

    # Generación de respuesta basada en el texto proporcionado
    with generate_lock:
        historico, output = generate_long_chat(historico, ai, user, input_text=entrada, max_additional_tokens=2048, short_answer=short_answer, streaming=False, printing=False)

    # si el idioma es español, traduce la respuesta al español


    return jsonify(entrada=texto, entrada_traducida="", respuesta=output, respuesta_traducida="")



import torch  # Importamos PyTorch para poder usar la función `to()`

# Función para mover recursivamente todos los tensores en una estructura anidada a un dispositivo
def move_to_device(obj, device):
    if isinstance(obj, torch.Tensor):
        return obj.to(device)
    elif isinstance(obj, dict):
        return {key: move_to_device(value, device) for key, value in obj.items()}
    elif isinstance(obj, list):
        return [move_to_device(item, device) for item in obj]
    else:
        return obj


# PREPARAMOS INSTANCIAS DE OpenVoice

if idioma == "en":

    from fairseq.checkpoint_utils import load_model_ensemble_and_task_from_hf_hub
    from fairseq.models.text_to_speech.hub_interface import TTSHubInterface

    # Carga el modelo y la configuración
    models, cfg, task = load_model_ensemble_and_task_from_hf_hub(
        "facebook/fastspeech2-en-ljspeech",
        arg_overrides={"vocoder": "hifigan", "fp16": False}
    )

    # Asegúrate de que models es una lista
    if not isinstance(models, list):
        models = [models]

    model = models[0]
    model = model.to('cuda:0')

    TTSHubInterface.update_cfg_with_data_cfg(cfg, task.data_cfg)

    # Aquí, asumimos que task.build_generator puede manejar correctamente el objeto cfg y model
    generator = task.build_generator(models, cfg)



elif idioma == "es":
    # El modelo no entiende de números aritméticos. Esta función los convierte a palabras.
    import re
    from num2words import num2words

    def number_to_words(num_str):
        try:
            num = int(num_str)
            return num2words(num, lang='es')
        except ValueError:
            return "Por favor, introduzca un número válido."

    def process_numbers_in_line(line):
        def replace_with_words(match):
            return number_to_words(match.group())

        return re.sub(r'\b\d+\b', replace_with_words, line)

    # Ejemplo de uso
    line = "Tengo 3 manzanas y 15 naranjas, sumando un total de 18 frutas."
    new_line = process_numbers_in_line(line)
    print(new_line)
    # Salida: "Tengo tres manzanas y quince naranjas, sumando un total de dieciocho frutas."


    # Diccionario con las traducciones

    def process_abrev(line):
        translations = {
        'Dr': 'doctor',
        'Sr': 'señor',
        'Sra': 'señora',
        # Añade más traducciones aquí
    }
        for abbr, full in translations.items():
            line = line.replace(f'{abbr}.', full)
            line = line.replace(f'{abbr} ', f'{full} ')
        return line

    def otras_traducciones(line):

        translations = {
        '-': ',',
        '—': ',',
        '%': ' por ciento '
        # Añade más traducciones aquí
        }

        for old, new in translations.items():
            line = line.replace(old, new)
        return line


    def preprocesado_al_modelo(line):
        line_with_numbers = process_numbers_in_line(line)
        line_with_both = process_abrev(line_with_numbers)
        line_with_all = otras_traducciones(line_with_both)
        return line_with_all

    import os
    os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'
    print(os.environ['TF_ENABLE_ONEDNN_OPTS'])


    from pydub import AudioSegment, silence

    def quitar_silencios(input_filepath, output_filepath, min_silence_len=1500, new_silence_len=750, silence_thresh=-60):
        """
        Elimina silencios largos de un archivo de audio.

        Parámetros:
        - input_filepath: ruta al archivo de audio de entrada (MP3).
        - output_filepath: ruta al archivo de audio de salida (MP3).
        - min_silence_len: duración mínima del silencio a eliminar (en milisegundos).
        - new_silence_len: duración de los nuevos segmentos de silencio (en milisegundos).
        - silence_thresh: umbral de silencio (en dB).
        """

        # Cargar el archivo de audio
        audio_segment = AudioSegment.from_wav(input_filepath)

        # Encuentra los segmentos de audio separados por silencios
        segments = silence.split_on_silence(audio_segment, min_silence_len=min_silence_len, silence_thresh=silence_thresh)

        # Crear un nuevo segmento de audio con silencios ajustados
        new_audio_segment = AudioSegment.empty()
        silence_chunk = AudioSegment.silent(duration=new_silence_len)  # Chunk de silencio de la nueva duración

        # Añade cada segmento de audio al nuevo audio, intercalando con los nuevos segmentos de silencio
        for segment in segments:
            new_audio_segment += segment + silence_chunk

        # Removemos el último chunk de silencio añadido
        new_audio_segment = new_audio_segment[:-new_silence_len]

        # Guarda el nuevo archivo de audio
        new_audio_segment.export(output_filepath, format="wav")


    # Cargamos el modelo generador fairseq
    from fairseq.checkpoint_utils import load_model_ensemble_and_task_from_hf_hub
    from fairseq.models.text_to_speech.hub_interface import TTSHubInterface
    import IPython.display as ipd





    # Cargamos el modelo y la configuración desde el modelo preentrenado de Hugging Face
    models, cfg, task = load_model_ensemble_and_task_from_hf_hub(
        "facebook/tts_transformer-es-css10",
        arg_overrides={"vocoder": "hifigan", "fp16": False}
    )
    model = models[0]

    # Movemos el modelo al dispositivo GPU
    model = model.to('cuda:0')

    # Actualizamos la configuración con los datos del task
    TTSHubInterface.update_cfg_with_data_cfg(cfg, task.data_cfg)

    # Creamos el generador
    generator = task.build_generator([model], cfg)


    import torchaudio

    import re

    def dividir_texto_con_minimo_palabras(texto, min_palabras=8):
        partes = re.split(r'([.,;:?!])', texto)
        partes_filtradas = [parte.strip() for parte in partes if parte.strip()]
        partes_combinadas = []
        parte_actual = ''

        for parte in partes_filtradas:
            if parte in '.,;:?!':
                parte_actual += parte
                if len(parte_actual.split()) >= min_palabras:
                    partes_combinadas.append(parte_actual)
                    parte_actual = ''
                else:
                    parte_actual += ' '
            else:
                parte_actual += parte + ' '

        if len(parte_actual.strip()) > 10:
            partes_combinadas.append(parte_actual.strip())

        return partes_combinadas

    def combinar_audios(audios_temporales):
        audio_combinado = "audio_combinado.wav"
        # Cargar el primer archivo de audio para inicializar la concatenación
        wav_total, rate = torchaudio.load(audios_temporales[0])

        # Iterar sobre los archivos restantes y concatenarlos
        for archivo in audios_temporales[1:]:
            wav, _ = torchaudio.load(archivo)
            wav_total = torch.cat((wav_total, wav), 1)

        # Guardar el audio combinado en un archivo final
        torchaudio.save(audio_combinado, wav_total, rate)

        return audio_combinado

    def voz_sintetica_spanish(text):
        text = preprocesado_al_modelo(text)

        lista_dividida = dividir_texto_con_minimo_palabras(text)

        audios_temporales = []

        for parte in lista_dividida:
            # Preparamos los datos de entrada para el modelo
            sample = TTSHubInterface.get_model_input(task, parte)

            # Movemos los datos al dispositivo GPU
            sample = move_to_device(sample, 'cuda:0')

            # Realizamos la predicción
            wav, rate = TTSHubInterface.get_prediction(task, model, generator, sample)

            if len(wav.shape) == 1:
                wav = wav.unsqueeze(0)

            # temp_file_name = "Temporal.wav"
            temp_file_name = f"temporal_{parte[:10]}.wav"
            torchaudio.save(temp_file_name, wav.to('cpu'), rate)
            audios_temporales.append(temp_file_name)

            combinado = combinar_audios(audios_temporales)
            sin_silencios = "sin_silencios.wav"
            # quitamos silencios
            quitar_silencios(combinado, sin_silencios, min_silence_len=1500, new_silence_len=750, silence_thresh=-60)


        with open(sin_silencios, "rb") as audio_file:
            audio_base64 = base64.b64encode(audio_file.read()).decode('utf-8')

        return audio_base64



import base64
@app.route('/audio', methods=['POST'])
def generate_audio():
    texto = request.json.get('texto', '')

    if not texto:
        return jsonify(error="No se proporcionó texto"), 400

    if idioma == "en":
        audio_base64 = voz_sintetica_english(texto)
        return jsonify(audio_base64=audio_base64)
    elif idioma == "es":
        audio_base64 = voz_sintetica_spanish(texto)
        return jsonify(audio_base64=audio_base64)

import base64
import io
import soundfile as sf


def add_comma_after_punctuation(text: str) -> str:
    # Lista de caracteres después de los cuales se debe agregar una coma
    punctuation_marks = ['.', '!', '?', '(', ')', ':']
    
    # Recorre cada marca de puntuación y añade una coma después de cada ocurrencia
    for mark in punctuation_marks:
        text = text.replace(mark, mark + ',')
    
    return text

# Ejemplo de uso de la función
#example_text = "Hello! How are you? I hope you're doing well. Let's meet tomorrow."
#modified_text = add_comma_after_punctuation(example_text)
#print(modified_text)

import io
import base64
import soundfile as sf
import os
import threading
from pydub import AudioSegment
import subprocess



def voz_sintetica_english(texto):
    texto = add_comma_after_punctuation(texto) #preprocesamos para mejora del modelo
    # Preparamos los datos de entrada para el modelo
    sample = TTSHubInterface.get_model_input(task, texto)

    # Movemos los datos al dispositivo GPU
    sample = move_to_device(sample, 'cuda:0')

    # Realizamos la predicción
    wav, rate = TTSHubInterface.get_prediction(task, model, generator, sample)


        # Convertimos el tensor wav a un buffer de audio en memoria y luego a un archivo temporal
    temp_wav_path = f"temp_synth_audio_{int(time.time() * 1000)}.wav"
    with io.BytesIO() as audio_buffer:
        sf.write(audio_buffer, wav.cpu().numpy(), rate, format='WAV')
        audio_buffer.seek(0)  # Regresamos al inicio del buffer para leerlo
        # Guardar en un archivo temporal
        with open(temp_wav_path, 'wb') as f:
            f.write(audio_buffer.read())

    # Añadir el audio al archivo de conversación en segundo plano
    add_audio_to_conversation_async(temp_wav_path, convert_to_mp3=True)  # Asegúrate de implementar la conversión dentro de esta función si es necesario

    # Convertir el buffer a base64 para retornar
    with open(temp_wav_path, 'rb') as f:
        audio_base64 = base64.b64encode(f.read()).decode('utf-8')


    return audio_base64


def print_routes(app):
    print("Endpoints disponibles:")
    for rule in app.url_map.iter_rules():
        methods = ','.join(sorted(rule.methods))
        print(f"{rule.endpoint}: {rule.rule} [{methods}]")


if __name__ == '__main__':
    print_routes(app)
    app.run(host='0.0.0.0', port=5500, threaded=True)


Overwriting server_conversacional.py


### Servidor streaming de generación de texto y voz sintética (SIN INDICES DE COLA).

In [7]:
%%writefile server_conversacional.py
from flask import Flask, request, jsonify, send_file
from flask_cors import CORS
from threading import Lock
import threading
import os
import torch
from pydub import AudioSegment
import pandas as pd
import random

import time


import whisper
modelWhisper = whisper.load_model('medium')


modelo = "zypher"

# parts = []  # lista de partes del texto
# generando = False

if modelo == "mistral":
    from modelo_mistral_base import generate_in_file_parts, load_model
elif modelo == "zypher":
    from modelo_Zypher_beta import generate_in_file_parts, pre_warm_chat, load_model, estado_generacion
else:
    print("modelo no encontrado")
    exit()

ai = "assistant"
user = "user"

contexto = """

"""

system_prompt = """
You are a kind and helpful assistan bot. You are here to help the user to find the best answer to his question.
"""

saludo = "Hello, I am ready to receive and process your input."

idioma = "en"

import sys

# Verifica si el comando tenía flag -s o --short
if "-s" in sys.argv or "--short" in sys.argv:
    short_answer = True

# Si encuentra el flag -es cambia el idioma a español
if "-es" in sys.argv:
    idioma = "es"

# Filtra los argumentos para eliminar los flags
args = [arg for arg in sys.argv[1:] if arg not in ["-s", "--short", "-es"]]

# Asigna los valores a system_prompt y saludo basándose en los argumentos restantes
if len(args) > 0:
    system_prompt = args[0]
if len(args) > 1:
    saludo = args[1]

if modelo == "mistral":
    historico = f"<|im_start|>system\n{system_prompt}<|im_end|>\n<|im_start|>assistant\n{saludo}<|im_end|>\n"
elif modelo == "zypher":
    historico = f"<|system|>{system_prompt}</s>\n<|assistant|>\n{saludo}</s>\n"


# load model
load_model(user=user, ai=ai)

print(f"{ai}:", saludo)

# Crea un bloqueo para proteger el código contra la concurrencia a la hora de transcribir
transcribe_lock = Lock()

generate_lock = Lock()

app = Flask(__name__)
# app.config['MAX_CONTENT_LENGTH'] = 30 * 1024 * 1024  # 30 MB

CORS(app)

output = ""

@app.route('/inicio', methods=['POST'])
def print_strings():
    global modelo
    global historico

    # Lee el archivo CSV y selecciona un personaje de ficción al azar
    def elegir_personaje_aleatorio():
        df = pd.read_csv('Personajes_ficcion.csv')
        return random.choice(df.iloc[:, 0].tolist())

    # Obtiene los datos del cuerpo de la solicitud
    data = request.json

    # Extrae los strings del objeto JSON
    system_prompt = data.get('system_prompt')
    saludo = data.get('saludo')

    # Preprocesamiento para reemplazar "#personaje" con un personaje aleatorio
    if "#personaje" in system_prompt:
        personaje_aleatorio = elegir_personaje_aleatorio()
        system_prompt = system_prompt.replace("#personaje", personaje_aleatorio)

    # Preprocesamiento para reemplazar "#personaje" con un personaje aleatorio en el saludo
    if "#personaje" in saludo:
        saludo = saludo.replace("#personaje", personaje_aleatorio)

    # Imprime los strings en el log del servidor
    print("INICIALIZANDO CONVERSACIÓN")
    conversation_file = 'conversacion.mp3'
    # si existe el archivo de conversación, lo elimina
    if os.path.exists(conversation_file):
        os.remove(conversation_file)
        
    print(f"system: {system_prompt}, saludo: {saludo}")

    if modelo == "mistral":
        historico = f"system\n{system_prompt}\nassistant\n{saludo}\n"
    elif modelo == "zypher":
        historico = f"{system_prompt}</s>\n\n{saludo}</s>\n"

    pre_warm_chat(historico)

    # Retorna una respuesta para indicar que se recibieron y procesaron los datos
    return jsonify({"message": saludo}), 200



@app.route('/all_conversation', methods=['GET'])
def all_conversation():
    # Asegúrate de que el path al archivo sea correcto para tu estructura de proyecto
    filepath = 'conversacion.mp3'
    if not os.path.exists(filepath):
        return jsonify(error="Archivo de conversación no encontrado"), 404

    # Leer el archivo y convertirlo a base64
    with open(filepath, 'rb') as audio_file:
        audio_base64 = base64.b64encode(audio_file.read()).decode('utf-8')

    return jsonify(audio_base64=audio_base64)



import subprocess

def convert_ogg_to_mp3(source_ogg_path, target_mp3_path):
    """
    Utiliza ffmpeg para convertir un archivo .ogg a .mp3.
    """
    command = ['ffmpeg', '-y' ,'-i', source_ogg_path, target_mp3_path]
    process = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

    # Si el comando falla, imprime la salida de error
    if process.returncode != 0:
        print(f"Error al convertir {source_ogg_path} a {target_mp3_path}")
        print("Salida de error de ffmpeg:")
        print(process.stderr.decode())




def convert_wav_to_mp3(source_wav_path, target_mp3_path):
    command = ['ffmpeg', '-i', source_wav_path, target_mp3_path]
    subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE)


def add_audio_to_conversation_async(source_path, convert_to_mp3=False):
    def task():
        if convert_to_mp3:
            # Convertir de WAV a MP3 si es necesario
            temp_mp3_path = source_path.replace('.wav', '.mp3')
            convert_wav_to_mp3(source_path, temp_mp3_path)
            final_path = temp_mp3_path
        else:
            final_path = source_path

        # Añadir al archivo de conversación
        sound = AudioSegment.from_file(final_path)
        conversation_file = 'conversacion.mp3'
        if os.path.exists(conversation_file):
            conversation_audio = AudioSegment.from_mp3(conversation_file)
            combined_audio = conversation_audio + sound
        else:
            combined_audio = sound
        combined_audio.export(conversation_file, format='mp3')
        
        # Limpiar archivos temporales
        os.remove(source_path)
        if convert_to_mp3:
            os.remove(temp_mp3_path)

    thread = threading.Thread(target=task)
    thread.start()



def generate_chat_background(entrada, phistorico, ai, user, short_answer):
    global output  # Indicar que se utilizará la variable global 'output'
    start_generation_time = time.time()
    # Ejecutar la generación de chat en un hilo aparte
    historico_local, output_local = generate_in_file_parts(phistorico, ai, user, input_text=entrada, max_additional_tokens=2048, short_answer=short_answer, streaming=True, printing=False)
    end_generation_time = time.time()
    generation_duration = end_generation_time - start_generation_time
    print(f"Generación completada en {generation_duration} segundos")
    
    # Actualizar las variables globales con los resultados obtenidos
    global historico
    historico = historico_local
    output = output_local

@app.route('/transcribe', methods=['POST'])
def transcribe_audio():
    print("Transcribiendo audio...")
    global historico
    global user
    global ai

    if 'file' not in request.files:
        return jsonify(error="No file part"), 400

    file = request.files['file']
    if file.filename == '':
        return jsonify(error="No selected file"), 400

    timestamp = int(time.time() * 1000)
    ogg_filepath = f"received_audio_{timestamp}.ogg"
    file.save(ogg_filepath)

    start_transcribe_time = time.time()
    with transcribe_lock:
        transcripcion = modelWhisper.transcribe(ogg_filepath, fp16=False, language=idioma)
        transcripcion = transcripcion["text"]
    end_transcribe_time = time.time()
    transcribe_duration = end_transcribe_time - start_transcribe_time
    print(f"Transcripción completada en {transcribe_duration} segundos")

    print("transcripción:", transcripcion)

    # Iniciar la generación de chat en un hilo aparte
    thread = threading.Thread(target=generate_chat_background, args=(transcripcion, historico, ai, user, short_answer))
    thread.start()

    # Inicia el proceso de adición del audio .ogg en segundo plano, considerando su conversión a .mp3
    add_audio_to_conversation_async(ogg_filepath)

    # La respuesta ya no incluirá 'output' porque se generará en segundo plano
    return jsonify(entrada=transcripcion, entrada_traducida="")



# from modelo_Zypher_beta import estado_generacion
# estado_lock = threading.Lock()

@app.route('/get_next_part', methods=['GET'])
def get_next_part():
    global estado_generacion

    print(f"partes: {estado_generacion.parts}, generando: {estado_generacion.generando}")

    while True:
        if estado_generacion.parts:
            with estado_generacion.lock:  # Asegurarse de que el acceso a 'parts' es seguro
                part = estado_generacion.parts.pop(0)  # Obtiene y elimina el primer elemento de la lista
                print(f"Enviando parte: {part}")
            return jsonify(output=part)
        elif estado_generacion.generando:
            print("Esperando a que se generen más partes...")
            time.sleep(0.1)  # Espera 0.1 segundos antes de volver a verificar
        else:
            print("No hay más partes para enviar")
            return jsonify(output="")  # Si 'generando' es False y 'parts' está vacía, devuelve una cadena vacía

# if __name__ == '__main__':
#     app.run(debug=True)


@app.route('/texto', methods=['POST'])
def process_text():
    global historico
    global user
    global ai

    # Recibe el texto directamente del cuerpo de la solicitud
    data = request.json
    if not data or 'texto' not in data:
        return jsonify(error="No se proporcionó texto"), 400

    texto = data['texto']

    # Utiliza la variable 'idioma' declarada globalmente
    global idioma

    entrada = texto

    # Generación de respuesta basada en el texto proporcionado
    thread = threading.Thread(target=generate_chat_background, args=(entrada, historico, ai, user, short_answer))
    thread.start()


    # si el idioma es español, traduce la respuesta al español


    return jsonify(entrada=texto, entrada_traducida="")



import torch  # Importamos PyTorch para poder usar la función `to()`

# Función para mover recursivamente todos los tensores en una estructura anidada a un dispositivo
def move_to_device(obj, device):
    if isinstance(obj, torch.Tensor):
        return obj.to(device)
    elif isinstance(obj, dict):
        return {key: move_to_device(value, device) for key, value in obj.items()}
    elif isinstance(obj, list):
        return [move_to_device(item, device) for item in obj]
    else:
        return obj


# PREPARAMOS INSTANCIAS DE OpenVoice

if idioma == "en":

    from fairseq.checkpoint_utils import load_model_ensemble_and_task_from_hf_hub
    from fairseq.models.text_to_speech.hub_interface import TTSHubInterface

    # Carga el modelo y la configuración
    models, cfg, task = load_model_ensemble_and_task_from_hf_hub(
        "facebook/fastspeech2-en-ljspeech",
        arg_overrides={"vocoder": "hifigan", "fp16": False}
    )

    # Asegúrate de que models es una lista
    if not isinstance(models, list):
        models = [models]

    model = models[0]
    model = model.to('cuda:0')

    TTSHubInterface.update_cfg_with_data_cfg(cfg, task.data_cfg)

    # Aquí, asumimos que task.build_generator puede manejar correctamente el objeto cfg y model
    generator = task.build_generator(models, cfg)



elif idioma == "es":
    # El modelo no entiende de números aritméticos. Esta función los convierte a palabras.
    import re
    from num2words import num2words

    def number_to_words(num_str):
        try:
            num = int(num_str)
            return num2words(num, lang='es')
        except ValueError:
            return "Por favor, introduzca un número válido."

    def process_numbers_in_line(line):
        def replace_with_words(match):
            return number_to_words(match.group())

        return re.sub(r'\b\d+\b', replace_with_words, line)

    # Ejemplo de uso
    line = "Tengo 3 manzanas y 15 naranjas, sumando un total de 18 frutas."
    new_line = process_numbers_in_line(line)
    print(new_line)
    # Salida: "Tengo tres manzanas y quince naranjas, sumando un total de dieciocho frutas."


    # Diccionario con las traducciones

    def process_abrev(line):
        translations = {
        'Dr': 'doctor',
        'Sr': 'señor',
        'Sra': 'señora',
        # Añade más traducciones aquí
    }
        for abbr, full in translations.items():
            line = line.replace(f'{abbr}.', full)
            line = line.replace(f'{abbr} ', f'{full} ')
        return line

    def otras_traducciones(line):

        translations = {
        '-': ',',
        '—': ',',
        '%': ' por ciento '
        # Añade más traducciones aquí
        }

        for old, new in translations.items():
            line = line.replace(old, new)
        return line


    def preprocesado_al_modelo(line):
        line_with_numbers = process_numbers_in_line(line)
        line_with_both = process_abrev(line_with_numbers)
        line_with_all = otras_traducciones(line_with_both)
        return line_with_all

    import os
    os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'
    print(os.environ['TF_ENABLE_ONEDNN_OPTS'])


    from pydub import AudioSegment, silence

    def quitar_silencios(input_filepath, output_filepath, min_silence_len=1500, new_silence_len=750, silence_thresh=-60):
        """
        Elimina silencios largos de un archivo de audio.

        Parámetros:
        - input_filepath: ruta al archivo de audio de entrada (MP3).
        - output_filepath: ruta al archivo de audio de salida (MP3).
        - min_silence_len: duración mínima del silencio a eliminar (en milisegundos).
        - new_silence_len: duración de los nuevos segmentos de silencio (en milisegundos).
        - silence_thresh: umbral de silencio (en dB).
        """

        # Cargar el archivo de audio
        audio_segment = AudioSegment.from_wav(input_filepath)

        # Encuentra los segmentos de audio separados por silencios
        segments = silence.split_on_silence(audio_segment, min_silence_len=min_silence_len, silence_thresh=silence_thresh)

        # Crear un nuevo segmento de audio con silencios ajustados
        new_audio_segment = AudioSegment.empty()
        silence_chunk = AudioSegment.silent(duration=new_silence_len)  # Chunk de silencio de la nueva duración

        # Añade cada segmento de audio al nuevo audio, intercalando con los nuevos segmentos de silencio
        for segment in segments:
            new_audio_segment += segment + silence_chunk

        # Removemos el último chunk de silencio añadido
        new_audio_segment = new_audio_segment[:-new_silence_len]

        # Guarda el nuevo archivo de audio
        new_audio_segment.export(output_filepath, format="wav")


    # Cargamos el modelo generador fairseq
    from fairseq.checkpoint_utils import load_model_ensemble_and_task_from_hf_hub
    from fairseq.models.text_to_speech.hub_interface import TTSHubInterface
    import IPython.display as ipd





    # Cargamos el modelo y la configuración desde el modelo preentrenado de Hugging Face
    models, cfg, task = load_model_ensemble_and_task_from_hf_hub(
        "facebook/tts_transformer-es-css10",
        arg_overrides={"vocoder": "hifigan", "fp16": False}
    )
    model = models[0]

    # Movemos el modelo al dispositivo GPU
    model = model.to('cuda:0')

    # Actualizamos la configuración con los datos del task
    TTSHubInterface.update_cfg_with_data_cfg(cfg, task.data_cfg)

    # Creamos el generador
    generator = task.build_generator([model], cfg)


    import torchaudio

    import re

    def dividir_texto_con_minimo_palabras(texto, min_palabras=8):
        partes = re.split(r'([.,;:?!])', texto)
        partes_filtradas = [parte.strip() for parte in partes if parte.strip()]
        partes_combinadas = []
        parte_actual = ''

        for parte in partes_filtradas:
            if parte in '.,;:?!':
                parte_actual += parte
                if len(parte_actual.split()) >= min_palabras:
                    partes_combinadas.append(parte_actual)
                    parte_actual = ''
                else:
                    parte_actual += ' '
            else:
                parte_actual += parte + ' '

        if len(parte_actual.strip()) > 10:
            partes_combinadas.append(parte_actual.strip())

        return partes_combinadas

    def combinar_audios(audios_temporales):
        audio_combinado = "audio_combinado.wav"
        # Cargar el primer archivo de audio para inicializar la concatenación
        wav_total, rate = torchaudio.load(audios_temporales[0])

        # Iterar sobre los archivos restantes y concatenarlos
        for archivo in audios_temporales[1:]:
            wav, _ = torchaudio.load(archivo)
            wav_total = torch.cat((wav_total, wav), 1)

        # Guardar el audio combinado en un archivo final
        torchaudio.save(audio_combinado, wav_total, rate)

        return audio_combinado

    def voz_sintetica_spanish(text):
        text = preprocesado_al_modelo(text)

        lista_dividida = dividir_texto_con_minimo_palabras(text)

        audios_temporales = []

        for parte in lista_dividida:
            # Preparamos los datos de entrada para el modelo
            sample = TTSHubInterface.get_model_input(task, parte)

            # Movemos los datos al dispositivo GPU
            sample = move_to_device(sample, 'cuda:0')

            # Realizamos la predicción
            wav, rate = TTSHubInterface.get_prediction(task, model, generator, sample)

            if len(wav.shape) == 1:
                wav = wav.unsqueeze(0)

            # temp_file_name = "Temporal.wav"
            temp_file_name = f"temporal_{parte[:10]}.wav"
            torchaudio.save(temp_file_name, wav.to('cpu'), rate)
            audios_temporales.append(temp_file_name)

            combinado = combinar_audios(audios_temporales)
            sin_silencios = "sin_silencios.wav"
            # quitamos silencios
            quitar_silencios(combinado, sin_silencios, min_silence_len=1500, new_silence_len=750, silence_thresh=-60)


        with open(sin_silencios, "rb") as audio_file:
            audio_base64 = base64.b64encode(audio_file.read()).decode('utf-8')

        return audio_base64



import base64
@app.route('/audio', methods=['POST'])
def generate_audio():
    texto = request.json.get('texto', '')

    if not texto:
        return jsonify(error="No se proporcionó texto"), 400

    if idioma == "en":
        audio_base64 = voz_sintetica_english(texto)
        return jsonify(audio_base64=audio_base64)
    elif idioma == "es":
        audio_base64 = voz_sintetica_spanish(texto)
        return jsonify(audio_base64=audio_base64)

import base64
import io
import soundfile as sf


def add_comma_after_punctuation(text: str) -> str:
    # Lista de caracteres después de los cuales se debe agregar una coma
    punctuation_marks = ['.', '!', '?', '(', ')', ':']
    
    # Recorre cada marca de puntuación y añade una coma después de cada ocurrencia
    for mark in punctuation_marks:
        text = text.replace(mark, mark + ',')
    
    return text

# Ejemplo de uso de la función
#example_text = "Hello! How are you? I hope you're doing well. Let's meet tomorrow."
#modified_text = add_comma_after_punctuation(example_text)
#print(modified_text)

import io
import base64
import soundfile as sf
import os
import threading
from pydub import AudioSegment
import subprocess



def voz_sintetica_english(texto):
    texto = add_comma_after_punctuation(texto) #preprocesamos para mejora del modelo
    # Preparamos los datos de entrada para el modelo
    sample = TTSHubInterface.get_model_input(task, texto)

    # Movemos los datos al dispositivo GPU
    sample = move_to_device(sample, 'cuda:0')

    # Realizamos la predicción
    wav, rate = TTSHubInterface.get_prediction(task, model, generator, sample)


        # Convertimos el tensor wav a un buffer de audio en memoria y luego a un archivo temporal
    temp_wav_path = f"temp_synth_audio_{int(time.time() * 1000)}.wav"
    with io.BytesIO() as audio_buffer:
        sf.write(audio_buffer, wav.cpu().numpy(), rate, format='WAV')
        audio_buffer.seek(0)  # Regresamos al inicio del buffer para leerlo
        # Guardar en un archivo temporal
        with open(temp_wav_path, 'wb') as f:
            f.write(audio_buffer.read())

    # Añadir el audio al archivo de conversación en segundo plano
    add_audio_to_conversation_async(temp_wav_path, convert_to_mp3=True)  # Asegúrate de implementar la conversión dentro de esta función si es necesario

    # Convertir el buffer a base64 para retornar
    with open(temp_wav_path, 'rb') as f:
        audio_base64 = base64.b64encode(f.read()).decode('utf-8')


    return audio_base64


def print_routes(app):
    print("Endpoints disponibles:")
    for rule in app.url_map.iter_rules():
        methods = ','.join(sorted(rule.methods))
        print(f"{rule.endpoint}: {rule.rule} [{methods}]")


if __name__ == '__main__':
    print_routes(app)
    app.run(host='0.0.0.0', port=5500, threaded=True)


Overwriting server_conversacional.py


### Servidor Streaming con indices de cola

In [12]:
%%writefile server_conversacional.py
from flask import Flask, request, jsonify, send_file
from flask_cors import CORS
from threading import Lock
import threading
import os
import torch
from pydub import AudioSegment
import pandas as pd
import random

import time


import whisper
modelWhisper = whisper.load_model('medium')


modelo = "zypher"

# parts = []  # lista de partes del texto
# generando = False

if modelo == "mistral":
    from modelo_mistral_base import generate_in_file_parts, load_model
elif modelo == "zypher":
    from modelo_Zypher_beta import generate_in_file_parts, pre_warm_chat, load_model, estado_generacion
else:
    print("modelo no encontrado")
    exit()

ai = "assistant"
user = "user"

contexto = """

"""

system_prompt = """
You are a kind and helpful assistan bot. You are here to help the user to find the best answer to his question.
"""

saludo = "Hello, I am ready to receive and process your input."

idioma = "en"

import sys

# Verifica si el comando tenía flag -s o --short
if "-s" in sys.argv or "--short" in sys.argv:
    short_answer = True

# Si encuentra el flag -es cambia el idioma a español
if "-es" in sys.argv:
    idioma = "es"

# Filtra los argumentos para eliminar los flags
args = [arg for arg in sys.argv[1:] if arg not in ["-s", "--short", "-es"]]

# Asigna los valores a system_prompt y saludo basándose en los argumentos restantes
if len(args) > 0:
    system_prompt = args[0]
if len(args) > 1:
    saludo = args[1]

if modelo == "mistral":
    historico = f"<|im_start|>system\n{system_prompt}<|im_end|>\n<|im_start|>assistant\n{saludo}<|im_end|>\n"
elif modelo == "zypher":
    historico = f"<|system|>{system_prompt}</s>\n<|assistant|>\n{saludo}</s>\n"


# load model
load_model(user=user, ai=ai)

print(f"{ai}:", saludo)

# Crea un bloqueo para proteger el código contra la concurrencia a la hora de transcribir
transcribe_lock = Lock()

generate_lock = Lock()

app = Flask(__name__)
# app.config['MAX_CONTENT_LENGTH'] = 30 * 1024 * 1024  # 30 MB

CORS(app)

output = ""

@app.route('/inicio', methods=['POST'])
def print_strings():
    global modelo
    global historico

    # Lee el archivo CSV y selecciona un personaje de ficción al azar
    def elegir_personaje_aleatorio():
        df = pd.read_csv('Personajes_ficcion.csv')
        return random.choice(df.iloc[:, 0].tolist())

    # Obtiene los datos del cuerpo de la solicitud
    data = request.json

    # Extrae los strings del objeto JSON
    system_prompt = data.get('system_prompt')
    saludo = data.get('saludo')

    # Preprocesamiento para reemplazar "#personaje" con un personaje aleatorio
    if "#personaje" in system_prompt:
        personaje_aleatorio = elegir_personaje_aleatorio()
        system_prompt = system_prompt.replace("#personaje", personaje_aleatorio)

    # Preprocesamiento para reemplazar "#personaje" con un personaje aleatorio en el saludo
    if "#personaje" in saludo:
        saludo = saludo.replace("#personaje", personaje_aleatorio)

    # Imprime los strings en el log del servidor
    print("INICIALIZANDO CONVERSACIÓN")
    conversation_file = 'conversacion.mp3'
    # si existe el archivo de conversación, lo elimina
    if os.path.exists(conversation_file):
        os.remove(conversation_file)
        
    print(f"system: {system_prompt}, saludo: {saludo}")

    if modelo == "mistral":
        historico = f"system\n{system_prompt}\nassistant\n{saludo}\n"
    elif modelo == "zypher":
        historico = f"{system_prompt}</s>\n\n{saludo}</s>\n"

    pre_warm_chat(historico)

    # Retorna una respuesta para indicar que se recibieron y procesaron los datos
    return jsonify({"message": saludo}), 200



@app.route('/all_conversation', methods=['GET'])
def all_conversation():
    # Asegúrate de que el path al archivo sea correcto para tu estructura de proyecto
    filepath = 'conversacion.mp3'
    if not os.path.exists(filepath):
        return jsonify(error="Archivo de conversación no encontrado"), 404

    # Leer el archivo y convertirlo a base64
    with open(filepath, 'rb') as audio_file:
        audio_base64 = base64.b64encode(audio_file.read()).decode('utf-8')

    return jsonify(audio_base64=audio_base64)



import subprocess

def convert_ogg_to_mp3(source_ogg_path, target_mp3_path):
    """
    Utiliza ffmpeg para convertir un archivo .ogg a .mp3.
    """
    command = ['ffmpeg', '-y' ,'-i', source_ogg_path, target_mp3_path]
    process = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

    # Si el comando falla, imprime la salida de error
    if process.returncode != 0:
        print(f"Error al convertir {source_ogg_path} a {target_mp3_path}")
        print("Salida de error de ffmpeg:")
        print(process.stderr.decode())




def convert_wav_to_mp3(source_wav_path, target_mp3_path):
    command = ['ffmpeg', '-i', source_wav_path, target_mp3_path]
    subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE)


def add_audio_to_conversation_async(source_path, convert_to_mp3=False):
    def task():
        if convert_to_mp3:
            # Convertir de WAV a MP3 si es necesario
            temp_mp3_path = source_path.replace('.wav', '.mp3')
            convert_wav_to_mp3(source_path, temp_mp3_path)
            final_path = temp_mp3_path
        else:
            final_path = source_path

        # Añadir al archivo de conversación
        sound = AudioSegment.from_file(final_path)
        conversation_file = 'conversacion.mp3'
        if os.path.exists(conversation_file):
            conversation_audio = AudioSegment.from_mp3(conversation_file)
            combined_audio = conversation_audio + sound
        else:
            combined_audio = sound
        combined_audio.export(conversation_file, format='mp3')
        
        # Limpiar archivos temporales
        os.remove(source_path)
        if convert_to_mp3:
            os.remove(temp_mp3_path)

    thread = threading.Thread(target=task)
    thread.start()



def generate_chat_background(entrada, phistorico, ai, user, short_answer):
    global output  # Indicar que se utilizará la variable global 'output'
    start_generation_time = time.time()
    # Ejecutar la generación de chat en un hilo aparte
    historico_local, output_local = generate_in_file_parts(phistorico, ai, user, input_text=entrada, max_additional_tokens=2048, short_answer=short_answer, streaming=True, printing=False)
    end_generation_time = time.time()
    generation_duration = end_generation_time - start_generation_time
    print(f"Generación completada en {generation_duration} segundos")
    
    # Actualizar las variables globales con los resultados obtenidos
    global historico
    historico = historico_local
    output = output_local

@app.route('/transcribe', methods=['POST'])
def transcribe_audio():
    print("Transcribiendo audio...")
    global historico
    global user
    global ai

    if 'file' not in request.files:
        return jsonify(error="No file part"), 400

    file = request.files['file']
    if file.filename == '':
        return jsonify(error="No selected file"), 400

    timestamp = int(time.time() * 1000)
    ogg_filepath = f"received_audio_{timestamp}.ogg"
    file.save(ogg_filepath)

    start_transcribe_time = time.time()
    with transcribe_lock:
        transcripcion = modelWhisper.transcribe(ogg_filepath, fp16=False, language=idioma)
        transcripcion = transcripcion["text"]
    end_transcribe_time = time.time()
    transcribe_duration = end_transcribe_time - start_transcribe_time
    print(f"Transcripción completada en {transcribe_duration} segundos")

    print("transcripción:", transcripcion)

    # Iniciar la generación de chat en un hilo aparte
    thread = threading.Thread(target=generate_chat_background, args=(transcripcion, historico, ai, user, short_answer))
    thread.start()

    # Inicia el proceso de adición del audio .ogg en segundo plano, considerando su conversión a .mp3
    add_audio_to_conversation_async(ogg_filepath)

    # La respuesta ya no incluirá 'output' porque se generará en segundo plano
    return jsonify(entrada=transcripcion, entrada_traducida="")



# from modelo_Zypher_beta import estado_generacion
# estado_lock = threading.Lock()

@app.route('/get_next_part', methods=['GET'])
def get_next_part():
    global estado_generacion

    # Obtener el índice de la solicitud. Si no se proporciona, por defecto es None
    index = request.args.get('index', default=None, type=int)

    print(f"partes: {estado_generacion.parts}, generando: {estado_generacion.generando}, index: {index}, estado_generacion.top: {estado_generacion.top}")

    while True:
        # if estado_generacion.parts:
            # Verificar si el índice es válido
        if index is not None and index >= 0 and index <= estado_generacion.top:
            part = estado_generacion.parts[index]
            # with estado_generacion.lock:  # Asegurarse de que el acceso a 'parts' es seguro
            # part = ""
            # contador = 0
            # while part == "" and contador < 100:
            #     parte = estado_generacion.parts[index]
            #     if parte != "":
                #     part = parte
                #     estado_generacion.parts[index] = ""  # Elimina el elemento en el índice dado
                # else:
                #     time.sleep(0.1)
                #     contador += 1
            print(f"Enviando parte: {part}")
            return jsonify(output=part)
            # else:
            #     print("Índice inválido o fuera de límites")
            #     return jsonify(error="Índice inválido o fuera de límites"), 400
        elif estado_generacion.generando:
            print("Esperando a que se generen más partes...")
            time.sleep(0.1)  # Espera 0.1 segundos antes de volver a verificar
        else:
            print("No hay más partes para enviar")
            return jsonify(output="") # Si 'generando' es False y 'parts' está vacía, devuelve una cadena vacía



@app.route('/texto', methods=['POST'])
def process_text():
    global historico
    global user
    global ai

    # Recibe el texto directamente del cuerpo de la solicitud
    data = request.json
    if not data or 'texto' not in data:
        return jsonify(error="No se proporcionó texto"), 400

    texto = data['texto']

    # Utiliza la variable 'idioma' declarada globalmente
    global idioma

    entrada = texto

    # Generación de respuesta basada en el texto proporcionado
    thread = threading.Thread(target=generate_chat_background, args=(entrada, historico, ai, user, short_answer))
    thread.start()


    # si el idioma es español, traduce la respuesta al español


    return jsonify(entrada=texto, entrada_traducida="")



import torch  # Importamos PyTorch para poder usar la función `to()`

# Función para mover recursivamente todos los tensores en una estructura anidada a un dispositivo
def move_to_device(obj, device):
    if isinstance(obj, torch.Tensor):
        return obj.to(device)
    elif isinstance(obj, dict):
        return {key: move_to_device(value, device) for key, value in obj.items()}
    elif isinstance(obj, list):
        return [move_to_device(item, device) for item in obj]
    else:
        return obj


# PREPARAMOS INSTANCIAS DE OpenVoice

if idioma == "en":

    from fairseq.checkpoint_utils import load_model_ensemble_and_task_from_hf_hub
    from fairseq.models.text_to_speech.hub_interface import TTSHubInterface

    # Carga el modelo y la configuración
    models, cfg, task = load_model_ensemble_and_task_from_hf_hub(
        "facebook/fastspeech2-en-ljspeech",
        arg_overrides={"vocoder": "hifigan", "fp16": False}
    )

    # Asegúrate de que models es una lista
    if not isinstance(models, list):
        models = [models]

    model = models[0]
    model = model.to('cuda:0')

    TTSHubInterface.update_cfg_with_data_cfg(cfg, task.data_cfg)

    # Aquí, asumimos que task.build_generator puede manejar correctamente el objeto cfg y model
    generator = task.build_generator(models, cfg)



elif idioma == "es":
    # El modelo no entiende de números aritméticos. Esta función los convierte a palabras.
    import re
    from num2words import num2words

    def number_to_words(num_str):
        try:
            num = int(num_str)
            return num2words(num, lang='es')
        except ValueError:
            return "Por favor, introduzca un número válido."

    def process_numbers_in_line(line):
        def replace_with_words(match):
            return number_to_words(match.group())

        return re.sub(r'\b\d+\b', replace_with_words, line)

    # Ejemplo de uso
    line = "Tengo 3 manzanas y 15 naranjas, sumando un total de 18 frutas."
    new_line = process_numbers_in_line(line)
    print(new_line)
    # Salida: "Tengo tres manzanas y quince naranjas, sumando un total de dieciocho frutas."


    # Diccionario con las traducciones

    def process_abrev(line):
        translations = {
        'Dr': 'doctor',
        'Sr': 'señor',
        'Sra': 'señora',
        # Añade más traducciones aquí
    }
        for abbr, full in translations.items():
            line = line.replace(f'{abbr}.', full)
            line = line.replace(f'{abbr} ', f'{full} ')
        return line

    def otras_traducciones(line):

        translations = {
        '-': ',',
        '—': ',',
        '%': ' por ciento '
        # Añade más traducciones aquí
        }

        for old, new in translations.items():
            line = line.replace(old, new)
        return line


    def preprocesado_al_modelo(line):
        line_with_numbers = process_numbers_in_line(line)
        line_with_both = process_abrev(line_with_numbers)
        line_with_all = otras_traducciones(line_with_both)
        return line_with_all

    import os
    os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'
    print(os.environ['TF_ENABLE_ONEDNN_OPTS'])


    from pydub import AudioSegment, silence

    def quitar_silencios(input_filepath, output_filepath, min_silence_len=1500, new_silence_len=750, silence_thresh=-60):
        """
        Elimina silencios largos de un archivo de audio.

        Parámetros:
        - input_filepath: ruta al archivo de audio de entrada (MP3).
        - output_filepath: ruta al archivo de audio de salida (MP3).
        - min_silence_len: duración mínima del silencio a eliminar (en milisegundos).
        - new_silence_len: duración de los nuevos segmentos de silencio (en milisegundos).
        - silence_thresh: umbral de silencio (en dB).
        """

        # Cargar el archivo de audio
        audio_segment = AudioSegment.from_wav(input_filepath)

        # Encuentra los segmentos de audio separados por silencios
        segments = silence.split_on_silence(audio_segment, min_silence_len=min_silence_len, silence_thresh=silence_thresh)

        # Crear un nuevo segmento de audio con silencios ajustados
        new_audio_segment = AudioSegment.empty()
        silence_chunk = AudioSegment.silent(duration=new_silence_len)  # Chunk de silencio de la nueva duración

        # Añade cada segmento de audio al nuevo audio, intercalando con los nuevos segmentos de silencio
        for segment in segments:
            new_audio_segment += segment + silence_chunk

        # Removemos el último chunk de silencio añadido
        new_audio_segment = new_audio_segment[:-new_silence_len]

        # Guarda el nuevo archivo de audio
        new_audio_segment.export(output_filepath, format="wav")


    # Cargamos el modelo generador fairseq
    from fairseq.checkpoint_utils import load_model_ensemble_and_task_from_hf_hub
    from fairseq.models.text_to_speech.hub_interface import TTSHubInterface
    import IPython.display as ipd





    # Cargamos el modelo y la configuración desde el modelo preentrenado de Hugging Face
    models, cfg, task = load_model_ensemble_and_task_from_hf_hub(
        "facebook/tts_transformer-es-css10",
        arg_overrides={"vocoder": "hifigan", "fp16": False}
    )
    model = models[0]

    # Movemos el modelo al dispositivo GPU
    model = model.to('cuda:0')

    # Actualizamos la configuración con los datos del task
    TTSHubInterface.update_cfg_with_data_cfg(cfg, task.data_cfg)

    # Creamos el generador
    generator = task.build_generator([model], cfg)


    import torchaudio

    import re

    def dividir_texto_con_minimo_palabras(texto, min_palabras=8):
        partes = re.split(r'([.,;:?!])', texto)
        partes_filtradas = [parte.strip() for parte in partes if parte.strip()]
        partes_combinadas = []
        parte_actual = ''

        for parte in partes_filtradas:
            if parte in '.,;:?!':
                parte_actual += parte
                if len(parte_actual.split()) >= min_palabras:
                    partes_combinadas.append(parte_actual)
                    parte_actual = ''
                else:
                    parte_actual += ' '
            else:
                parte_actual += parte + ' '

        if len(parte_actual.strip()) > 10:
            partes_combinadas.append(parte_actual.strip())

        return partes_combinadas

    def combinar_audios(audios_temporales):
        audio_combinado = "audio_combinado.wav"
        # Cargar el primer archivo de audio para inicializar la concatenación
        wav_total, rate = torchaudio.load(audios_temporales[0])

        # Iterar sobre los archivos restantes y concatenarlos
        for archivo in audios_temporales[1:]:
            wav, _ = torchaudio.load(archivo)
            wav_total = torch.cat((wav_total, wav), 1)

        # Guardar el audio combinado en un archivo final
        torchaudio.save(audio_combinado, wav_total, rate)

        return audio_combinado

    def voz_sintetica_spanish(text):
        text = preprocesado_al_modelo(text)

        lista_dividida = dividir_texto_con_minimo_palabras(text)

        audios_temporales = []

        for parte in lista_dividida:
            # Preparamos los datos de entrada para el modelo
            sample = TTSHubInterface.get_model_input(task, parte)

            # Movemos los datos al dispositivo GPU
            sample = move_to_device(sample, 'cuda:0')

            # Realizamos la predicción
            wav, rate = TTSHubInterface.get_prediction(task, model, generator, sample)

            if len(wav.shape) == 1:
                wav = wav.unsqueeze(0)

            # temp_file_name = "Temporal.wav"
            temp_file_name = f"temporal_{parte[:10]}.wav"
            torchaudio.save(temp_file_name, wav.to('cpu'), rate)
            audios_temporales.append(temp_file_name)

            combinado = combinar_audios(audios_temporales)
            sin_silencios = "sin_silencios.wav"
            # quitamos silencios
            quitar_silencios(combinado, sin_silencios, min_silence_len=1500, new_silence_len=750, silence_thresh=-60)


        with open(sin_silencios, "rb") as audio_file:
            audio_base64 = base64.b64encode(audio_file.read()).decode('utf-8')

        return audio_base64



import base64
@app.route('/audio', methods=['POST'])
def generate_audio():
    texto = request.json.get('texto', '')

    if not texto:
        return jsonify(error="No se proporcionó texto"), 400

    if idioma == "en":
        audio_base64 = voz_sintetica_english(texto)
        return jsonify(audio_base64=audio_base64)
    elif idioma == "es":
        audio_base64 = voz_sintetica_spanish(texto)
        return jsonify(audio_base64=audio_base64)

import base64
import io
import soundfile as sf


def add_comma_after_punctuation(text: str) -> str:
    # Lista de caracteres después de los cuales se debe agregar una coma
    punctuation_marks = ['.', '!', '?', '(', ')', ':']
    
    # Recorre cada marca de puntuación y añade una coma después de cada ocurrencia
    for mark in punctuation_marks:
        text = text.replace(mark, mark + ',')
    
    return text

# Ejemplo de uso de la función
#example_text = "Hello! How are you? I hope you're doing well. Let's meet tomorrow."
#modified_text = add_comma_after_punctuation(example_text)
#print(modified_text)

import io
import base64
import soundfile as sf
import os
import threading
from pydub import AudioSegment
import subprocess



def voz_sintetica_english(texto):
    texto = add_comma_after_punctuation(texto) #preprocesamos para mejora del modelo
    # Preparamos los datos de entrada para el modelo
    sample = TTSHubInterface.get_model_input(task, texto)

    # Movemos los datos al dispositivo GPU
    sample = move_to_device(sample, 'cuda:0')

    # Realizamos la predicción
    wav, rate = TTSHubInterface.get_prediction(task, model, generator, sample)


        # Convertimos el tensor wav a un buffer de audio en memoria y luego a un archivo temporal
    temp_wav_path = f"temp_synth_audio_{int(time.time() * 1000)}.wav"
    with io.BytesIO() as audio_buffer:
        sf.write(audio_buffer, wav.cpu().numpy(), rate, format='WAV')
        audio_buffer.seek(0)  # Regresamos al inicio del buffer para leerlo
        # Guardar en un archivo temporal
        with open(temp_wav_path, 'wb') as f:
            f.write(audio_buffer.read())

    # Añadir el audio al archivo de conversación en segundo plano
    add_audio_to_conversation_async(temp_wav_path, convert_to_mp3=True)  # Asegúrate de implementar la conversión dentro de esta función si es necesario

    # Convertir el buffer a base64 para retornar
    with open(temp_wav_path, 'rb') as f:
        audio_base64 = base64.b64encode(f.read()).decode('utf-8')


    return audio_base64


def print_routes(app):
    print("Endpoints disponibles:")
    for rule in app.url_map.iter_rules():
        methods = ','.join(sorted(rule.methods))
        print(f"{rule.endpoint}: {rule.rule} [{methods}]")


if __name__ == '__main__':
    print_routes(app)
    app.run(host='0.0.0.0', port=5500, threaded=True)


Overwriting server_conversacional.py


## pm2 ejecutor de procesos en segundo plano

In [8]:
%%writefile server_conversacional.config.js
module.exports = {
  apps: [
    {
      name: "server_conversacional",
      script: "server_conversacional.py",
      args: ["--short", "-en", "You are Edward, the CEO of a consulting company called mAgIc, dedicated to finding artificial intelligence solutions for companies. You are interviewing a candidate to work for the company, you are interested in programming skills, mathematics, data science, AI and especially NLP. Ask questions about all this and probe. If the candidate says something wrong let them know. After 4 questions if most were good responses you must say 'CONGRATULATION YOU ARE IN OUR TEAM!!' if there was a bad response then you must say 'The interview is finished, thanks for come' IMPORTANT: if the interviewee is rude then you must say 'go away, close the door when go out'", "Good morning, wellcome to MAgIc, have a seat, tell me your name."],
      out_file: "out.log",
      error_file: "err.log",
      log_file: "combined.log",
      time: true,
    },
  ],
};


Overwriting server_conversacional.config.js


In [20]:
!pm2 start server_conversacional.config.js

[PM2][WARN] Applications server_conversacional not running, starting...
[PM2] App [server_conversacional] launched (1 instances)
┌────┬────────────────────┬──────────┬──────┬───────────┬──────────┬──────────┐
│ id │ name               │ mode     │ ↺    │ status    │ cpu      │ memory   │
├────┼────────────────────┼──────────┼──────┼───────────┼──────────┼──────────┤
│ 0  │ server_conversaci… │ fork     │ 0    │ online    │ 0%       │ 5.4mb    │
└────┴────────────────────┴──────────┴──────┴───────────┴──────────┴──────────┘


In [16]:
!tail -n 20 ./combined.log

2024-02-16T10:15:57: Game example
2024-02-16T10:15:57: (in this example your character is "superman")
2024-02-16T10:15:57: assistant: I'm thinking of a famous fictional character, guess which one he is.
2024-02-16T10:15:57: user: is it male or female?
2024-02-16T10:15:57: assistant: he is a man.
2024-02-16T10:15:57: user: do you have superpowers?
2024-02-16T10:15:57: assistant: Yes, he can fly, he has a lot of strength and is capable of projecting laser rays with his eyes.
2024-02-16T10:15:57: user: Is he superman?
2024-02-16T10:15:57: assistant: Congratulations, you were right, he was superman.
2024-02-16T10:15:57: </s>
2024-02-16T10:15:57: 
2024-02-16T10:15:57: I'm thinking of a famous fictional character, guess which one he is.</s>
2024-02-16T10:15:57: 
2024-02-16T10:15:57: user: Is it a man or a female?</s>
2024-02-16T10:15:57: assistant: He is a man.</s>
2024-02-16T10:15:57: user: Has he super powers?</s>
2024-02-16T10:15:57: assistant: Yes, he can fly, he has a lot of strength an

In [75]:
# !curl -F "file=@grabacion.ogg"  http://ia.javiergimenez.es:5500/transcribe
!curl -F "file=@grabacion.ogg"  http://localhost:5500/transcribe

{"entrada":" This is a walkie talkie test to see if it works, how does milk work?","entrada_traducida":"","respuesta":" Excuse me, I'm afraid that question doesn't seem relevant to this interview. Could you please provide an answer related to programming skills, mathematics, data science, AI or NLP instead? Our company, mAgIc, specializes in finding artificial intelligence solutions for businesses using these fields of expertise.","respuesta_traducida":""}


In [32]:
!curl -X POST http://localhost:5500/inicio \
     -H "Content-Type: application/json" \
     -d '{"system_prompt": "Eres un vendedor de enciclopedia", "saludo": "Hola buenos días, le gustaría comprar una enciclopedia?"}'


{"message":"Strings received and conversation initialized."}


In [10]:
!pm2 list

┌────┬────────────────────┬──────────┬──────┬───────────┬──────────┬──────────┐
│ id │ name               │ mode     │ ↺    │ status    │ cpu      │ memory   │
└────┴────────────────────┴──────────┴──────┴───────────┴──────────┴──────────┘


In [19]:
%%bash
pm2 stop 0
pm2 delete 0

[PM2] Applying action stopProcessId on app [0](ids: [ '0' ])


[PM2] [server_conversacional](0) ✓
┌────┬──────────────────────────┬─────────────┬─────────┬─────────┬──────────┬────────┬──────┬───────────┬──────────┬──────────┬──────────┬──────────┐
│ id │ name                     │ namespace   │ version │ mode    │ pid      │ uptime │ ↺    │ status    │ cpu      │ mem      │ user     │ watching │
├────┼──────────────────────────┼─────────────┼─────────┼─────────┼──────────┼────────┼──────┼───────────┼──────────┼──────────┼──────────┼──────────┤
│ 0  │ server_conversacional    │ default     │ N/A     │ fork    │ 0        │ 0      │ 0    │ stopped   │ 0%       │ 0b       │ javier   │ disabled │
└────┴──────────────────────────┴─────────────┴─────────┴─────────┴──────────┴────────┴──────┴───────────┴──────────┴──────────┴──────────┴──────────┘
[PM2] Applying action deleteProcessId on app [0](ids: [ '0' ])
[PM2] [server_conversacional](0) ✓
┌────┬───────────┬─────────────┬─────────┬─────────┬──────────┬────────┬──────┬───────────┬──────────┬──────────

## Cliente javascript

### intentando solucionar el problema del baile de la cola

In [22]:
%%writefile miniJuegoClienteBuffer.html

<head>
  <style>


    body {
        font-family: Arial, sans-serif; /* Mejora la tipografía general */
    }


    #system_prompt {
        height: 150px;
    }


    #inicioForm {
        max-width: 1000px; /* Limita el ancho del formulario */
        margin: 20px auto; /* Centra el formulario */
        padding: 20px;
        box-shadow: 0 0 10px rgba(0, 0, 0, 0.1); /* Añade un sombreado ligero */
    }

    #inicioForm div {
        margin-bottom: 15px; /* Añade más espacio entre los campos */
    }

    #inicioForm label {
        font-weight: bold; /* Hace que las etiquetas sean más notables */
        display: block; /* Asegura que la etiqueta esté encima del input */
        margin-bottom: 5px; /* Espacio entre la etiqueta y el campo */
    }

    #inicioForm select, #inicioForm textarea, #inicioForm button {
        width: 100%; /* Aprovecha todo el ancho disponible */
        padding: 8px; /* Añade un relleno para mayor comodidad */
        margin-top: 4px; /* Espacio mínimo superior para separación */
    }

    #inicioForm select {
        cursor: pointer; /* Indica que es un elemento interactivo */
        font-size: 16px; /* Aumenta el tamaño del texto */
    }

    #inicioForm textarea {
        resize: vertical; /* Permite al usuario ajustar la altura verticalmente */
    }

    #inicioForm button {
        background-color: #007bff; /* Color de fondo */
        color: white; /* Color del texto */
        border: none; /* Elimina el borde */
        padding: 10px 15px; /* Añade relleno */
        font-size: 18px; /* Aumenta el tamaño del texto */
        cursor: pointer; /* Indica que es un elemento interactivo */
        border-radius: 5px; /* Bordes redondeados */
    }

    #inicioForm button:hover {
        background-color: #0056b3; /* Oscurece el botón al pasar el mouse */
    }

#audioPlayerContainer {
    /* Añade estilos específicos si planeas insertar un reproductor de audio */
    margin-bottom: 20px; /* Espacio antes del botón de grabación */
}

#recordButton {
    background-color: #f44336; /* Color rojo para la grabación */
    color: white;
    border: none;
    padding: 10px 0;
    font-size: 18px;
    border-radius: 5px;
    cursor: pointer;
}

#recordButton:hover {
    background-color: #d32f2f; /* Oscurece el botón al pasar el mouse */
}

/* Estilos para el área de texto y botón de envío */
/* Contenedor del área de texto y el botón */
div#textButtonContainer {
    display: flex; /* Establece el contenedor para usar flexbox */
    justify-content: space-between; /* Espacia los elementos uniformemente */
    align-items: center; /* Alinea los elementos verticalmente en el centro */
}

/* Área de texto */
#textInput {
    flex-grow: 1; /* Permite que el área de texto crezca para ocupar el espacio disponible */
    margin-right: 10px; /* Añade un margen a la derecha para separarlo del botón */
    border: 1px solid #ccc; /* Establece un borde sutil */
    border-radius: 5px; /* Bordes redondeados */
    padding: 8px; /* Añade padding interno */
}

/* Botón */
#sendTextButton {
    padding: 8px 15px; /* Ajusta el padding para dimensionar el botón */
    background-color: #4CAF50; /* Color de fondo */
    color: white; /* Color del texto */
    border: none; /* Elimina el borde */
    border-radius: 5px; /* Bordes redondeados */
    cursor: pointer; /* Cambia el cursor a mano al pasar sobre el botón */
}

#sendTextButton:hover {
    background-color: #388E3C; /* Oscurece el botón al pasar el mouse */
}


#responseText {
    height: 250px;
    margin-top: 20px;
    border: 1px solid #ddd;
    padding: 10px;
    overflow-y: auto; /* Asegura el desplazamiento vertical */
    background-color: #f9f9f9; /* Fondo claro para resaltar el área */
}



</style>
</head>

<script>

document.addEventListener('DOMContentLoaded', function() {
    window.scrollTo(0, 0); // Asegura que la página comience en la parte superior
    document.getElementById('ejercicios').focus(); // Luego establece el foco en el selector
});



// Objeto para mapear los ejercicios a sus strings correspondientes
const ejerciciosStrings = {
    "guessing_game1": {
        systemPrompt: `This is a conversational game in which you have to think in this famous character: #personaje, and the user have to guess this character. You should aswer the user questions about the character. Be concise but give some clues. NEVER say the name of the character until the end. If the user guesses the character then you will say: "Congratulations, you got it right, the character was #personaje". If user give up you will say: "what a shame!, the character was #personaje
Game success example
assistant: I'm thinking of a famous fictional character, guess which one it is.
user: is it real or fictional?
assistant: it is fictional
user: Is he #personaje?
assistant: Congratulations, you were right, it was #personaje.
Game give up example
assistant: I'm thinking of a famous fictional character, guess which one it is.
user: is it real or fictional?
assistant: it is fictional
user: Is he Benito Perez?
assistant: No, it has nothing to do with.
user: I give up. Who is it?
assistant: what a shame!, the character was #personaje
`,
        saludo: "I'm thinking of a famous fictional character, guess which one it is."
    },
    "guessing_game2": {
        systemPrompt: "You are #personaje, the fictional character. You have to take on the personality of that character and engage in conversations about your events and experiences.",
        saludo: "I'm #personaje, the fictional character. Ask me anything you want to know about me."
    },    
    "guessing_game3": {
        systemPrompt: "This is a conversational game in which you have to guess a famous character. You should make questions to the user in order to guess the character that the user have choosen.",
        saludo: "Do you want to play? I will guess your choosen character by asking about it."
    },
    "yes_no_game": {
        systemPrompt: `IMPORTANT: You only can answer "yes" or "no" (nothing more!).
This is a conversational game between you and a the user. The game consists that only at the beggininig you tell the user only a piece of the context and the user having to guess the "key point" of the context from "yes or no questions". The User could ask anything about the story (context) to guess the "key point" of the context but Assistant could only answer "Yes" or "not". If the user asks a question that does not lend itself or cannot be answered with a "yes" or "no" such as "What is the man's name?" then Assistant will respond: "Only "yes or no" questions. When the user guesses the key point of the story you will say: "Congratulations, you have guessed the key to the story.
Game context: A man named Edgar is the lighthouse keeper of Águilas for 30 years. He always turns on the lighthouse at dusk and shortly after sleeps in a small room next to its large lamp. On Edgar's birthday, at dusk after lighting the lighthouse, he decided to go to dinner with an old friend to celebrate his birthday. During dinner he drank more than necessary and they both got drunk. Afther the dinner Edgar accompanied his friend to her house and then went to his house, the lighthouse, where he sleeps every night. Upon entering the lighthouse and going up to his room, due to his drunkenness and the fact that he was very sleepy, he decided to turn off the light (which was actually the light from the main lamp of the lighthouse) to sleep off the drunkenness and did it without realizing or knowing it was dangerous. During the early hours of the morning, a cruise ship full of passengers crashed into the cliff that the lighthouse protected because, when it was turned off, neither the lookout, nor the captain, nor the rest of the crew nor the passengers could see that they were heading against the cliff. An hour later Edgar wakes up, it hasn't dawned yet but you can hear sirens and a lot of noise from the rescuers who are trying to rescue the shipwrecked. Edgar turns on the lamp to illuminate the scene where hundreds of dead shipwrecked people continually crash against the cliff due to the waves. Faced with this heartbreaking reality and his feeling of guilt, Edgar decides to commit suicide by jumping from the top of the lighthouse. The key point of the story that the user must find out is: "Edgar commits suicide because he was the LIGHTHOUSE keeper." or similar but always emphasizing that he was the lighthouse keeper.
IMPORTANT: You only can answer "yes" or "no" or "Only yes or no questions"
Examples of correct answers:
user: What is Edgar's job?
Assistant: Only yes or no questions.
user: Is Edgar a man?
Assistant: yes.`,
        saludo: `This is I can show about the hidden story: Edgar was dazed and comes to his room, turns off the light and lies down on his bed. He wakes up a few hours later, turns on the light, looks out the window and is so horrified that he ends up jumping out of the window and committing suicide.
Guess what happened.
IMPORTANT: From now on I can only answer you "yes" or "no" and nothing more.`
    },
    "English_teacher": {
        systemPrompt: "You are an english teacher. You will have simple dialogues with the student in your charge. you will only have concise conversations with short sentences so that the student is encouraged to converse.",
        saludo: "Good Morning. What is your name?"
    }
    // Añade más ejercicios según sea necesario
};
</script>


<form id="inicioForm">
    <div>
        <label for="system_prompt">System Prompt:</label><br>
        <textarea id="system_prompt" name="system_prompt" rows="10" cols="100"></textarea>
    </div>
    <div>
        <label for="saludo">Saludo:</label><br>
        <textarea id="saludo" name="saludo" rows="4" cols="100"></textarea>
    </div>
    <div>
        <label for="ejercicios">Juegos:</label><br>
        <select id="ejercicios" name="ejercicios">
            <option value="">Selecciona un ejercicio</option>
            <option value="English_teacher">English teacher</option>
            <option value="guessing_game1">Guessing game 1</option>
            <option value="guessing_game2">Guessing game 2</option>
            <option value="guessing_game3">Guessing game 3</option>
            <option value="yes_no_game">yes no game</option>
        </select>
    </div>
    <button type="submit">EMPEZAR!</button>

</form>


<div style="margin-bottom: 20px; display: flex; align-items: center;"> 
    <button id="downloadButton" style="background-color: #4CAF50; /* Color de fondo */
                                       color: white; /* Color del texto */
                                       padding: 15px 32px; /* Padding alrededor del texto */
                                       text-align: center; /* Alinea el texto al centro */
                                       text-decoration: none; /* Elimina la decoración del texto */
                                       display: inline-block; /* Hace que el botón sea un bloque en línea */
                                       font-size: 16px; /* Tamaño del texto */
                                       margin: 4px 2px; /* Margen alrededor del botón */
                                       cursor: pointer; /* Cambia el cursor a un puntero */
                                       border: none; /* Elimina el borde */
                                       border-radius: 8px; /* Redondea las esquinas del botón */
    ">Descargar Conversación</button>
    <div id="audioPlayerAllContainer"></div>
</div>


<script>
    document.getElementById('downloadButton').addEventListener('click', function() {
        obtenerYReproducirAll(); // Llama a la función en vez de redirigir
    });
</script>


<script>

document.getElementById('ejercicios').addEventListener('change', function() {
    var selectedKey = this.value; // La clave seleccionada del objeto
    if (selectedKey) {
        // Actualiza los textareas con los valores correspondientes
        document.getElementById('system_prompt').value = ejerciciosStrings[selectedKey].systemPrompt;
        document.getElementById('saludo').value = ejerciciosStrings[selectedKey].saludo;
    }
});


document.getElementById('inicioForm').addEventListener('submit', function(e) {
    // Prevenir el comportamiento predeterminado del formulario
    e.preventDefault();

    // Obtener los valores de los campos del formulario
    const system_prompt = document.getElementById('system_prompt').value;
    const saludo = document.getElementById('saludo').value;

    document.getElementById('responseText').innerText = ""
    //obtenerYReproducirAudio(saludo)
    //updateResponseText(saludo + "\n")

    // Crear el cuerpo de la solicitud
    const data = { system_prompt, saludo };

    // Realizar la llamada al servicio Flask
    fetch('http://localhost:5500/inicio', {
        method: 'POST',
        headers: {
            'Content-Type': 'application/json',
        },
        body: JSON.stringify(data),
    })
    .then(response => response.json())
    .then(data => {
        let saludo = data.message;
        //document.getElementById('saludo').value = saludo;
        obtenerYReproducirAudio(saludo)
        updateResponseText(saludo + "\n")
        alert('Nuevo Juego iniciado');
    })
    .catch((error) => {
        console.error('Error:', error);
        alert('Error al enviar los strings');
    });
});
</script>


<div id="audioPlayerContainer"></div>
<button id="recordButton" style="width: 100%; height: 50px;">Pulsa para grabar/detener</button>


<!-- Estilos para textarea y botón de envío -->
<div id="textButtonContainer" style="margin-top: 10px;">
    <textarea id="textInput" placeholder="Escribe tu texto aquí" rows="4"></textarea>
    <button id="sendTextButton">Enviar Texto</button>
</div>



<div id="responseText" style="height: 250px; margin-top: 20px; border: 1px solid #ddd; padding: 10px; overflow-y:auto;"></div>
<script>
let recordButton = document.getElementById("recordButton");
let chunks = [];
let mediaRecorder;
let isRecording = false;

navigator.mediaDevices.getUserMedia({ audio: true })
.then(stream => {
    mediaRecorder = new MediaRecorder(stream);
    mediaRecorder.ondataavailable = event => {
        chunks.push(event.data);
    };
    mediaRecorder.onstop = () => {
        console.log('mediaRecorder Detenido!!');
        let blob = new Blob(chunks, { 'type': 'audio/ogg; codecs=opus' });
        enviarAudioAlServidor(blob);
        chunks = [];
    };
});

recordButton.onclick = () => {
    if (!isRecording) {
        mediaRecorder.start();
        isRecording = true;
        recordButton.textContent = 'Grabando...';
    } else {
        mediaRecorder.stop();
        isRecording = false;
        recordButton.textContent = 'Pulsa para grabar/detener';
    }
};


function enviarAudioAlServidor(blob) {
    let formData = new FormData();
    formData.append('file', blob, 'grabacion.ogg');

    fetch('http://localhost:5500/transcribe', {
        method: 'POST',
        body: formData,
    })
    .then(response => response.json())
    .then(data => {
        updateResponseText("\nyo: " + data.entrada + "\n\n" + "\n***********************************\n" + "respuesta: ");
        
        let indice = 0; // Inicializa el índice para las partes de la respuesta

        let obtenerPartes = () => {
            // Añade el índice como parámetro de la consulta a la URL
            fetch(`http://localhost:5500/get_next_part?index=${indice}`) // Cambia la URL según tu configuración
            .then(response => response.json())
            .then(partData => {
                if (partData.output !== "") { // Mientras la parte de la respuesta no esté vacía
                    let trozo = partData.output;
                    updateResponseText(trozo);
                    obtenerYReproducirAudio(trozo);

                    indice++; // Incrementa el índice para la próxima llamada
                    obtenerPartes(); // Vuelve a llamar a obtenerPartes recursivamente con el nuevo índice
                }
            })
            .catch(error => {
                console.error('Error al obtener la siguiente parte:', error);
            });
        };

        obtenerPartes(); // Comienza a obtener las partes de la respuesta
    })
    .catch(error => {
        console.error('Error al enviar el audio:', error);
    });
}



// Funcionalidad para enviar texto al servidor y limpiar el textarea
document.getElementById("sendTextButton").onclick = () => {
    let textInput = document.getElementById("textInput");
    let texto = textInput.value;
    if (texto) {
        enviarTextoAlServidor(texto);
        textInput.value = ''; // Limpiar el textarea después de enviar
    }
};

function enviarTextoAlServidor(texto) {
    fetch('http://localhost:5500/texto', { // Asegúrate de usar la URL correcta
        method: 'POST',
        headers: {
            'Content-Type': 'application/json'
        },
        body: JSON.stringify({ texto: texto })
    })
    .then(response => response.json())
    .then(data => {
        updateResponseText("\nyo: " + data.entrada + "\n\n" + "\n***********************************\n" + "respuesta: ");
        
        let indice = 0; // Inicializa el índice para las partes de la respuesta

        let obtenerPartes = () => {
            // Añade el índice como parámetro de la consulta
            fetch(`http://localhost:5500/get_next_part?index=${indice}`) 
            .then(response => response.json())
            .then(partData => {
                if (partData.output !== "") { // Mientras la parte de la respuesta no esté vacía
                    let trozo = partData.output;
                    updateResponseText(trozo);
                    obtenerYReproducirAudio(trozo);

                    indice++; // Incrementa el índice para la próxima llamada
                    obtenerPartes(); // Vuelve a llamar a obtenerPartes recursivamente con el nuevo índice
                }
            })
            .catch(error => {
                console.error('Error al obtener la siguiente parte:', error);
            });
        };

        obtenerPartes(); // Comienza a obtener las partes de la respuesta
    })
    .catch(error => {
        console.error('Error al enviar el texto:', error);
    });
}


function updateResponseText(text) {
    // document.getElementById('responseText').innerText += text;
    textarea = document.getElementById('responseText')
    textarea.innerText += text;
    textarea.scrollTop = textarea.scrollHeight;
}



function obtenerYReproducirAll() {
    fetch('http://localhost:5500/all_conversation', {
        method: 'GET',
    })
    .then(response => response.json())
    .then(data => {
        if (data.audio_base64) {
            createAudioAllPlayer(data.audio_base64);
        }
    })
    .catch(error => {
        console.error('Error al obtener el audio:', error);
    });
}


function createAudioAllPlayer(base64Audio) {
    let audioContainer = document.getElementById('audioPlayerAllContainer');
    let audioSrc = `data:audio/wav;base64,${base64Audio}`;
    let audioPlayer = document.createElement('audio');
    audioPlayer.src = audioSrc;
    audioPlayer.controls = true;
    audioPlayer.autoplay = true;
    audioContainer.innerHTML = '';
    audioContainer.appendChild(audioPlayer);
}




function obtenerYReproducirAudio(texto) {
    fetch('http://localhost:5500/audio', {

        method: 'POST',
        headers: {
            'Content-Type': 'application/json'
        },
        body: JSON.stringify({ texto: texto })
    })
    .then(response => response.json())
    .then(data => {
        if (data.audio_base64) {
            addAudioClipToQueue(data.audio_base64);
        }
    })
    .catch(error => {
        console.error('Error al obtener el audio:', error);
    });
}



let audioQueue = [];
let currentAudioIndex = 0; // Para controlar el orden de reproducción

function addAudioClipToQueue(base64Audio, index) {
    audioQueue.push({ audio: base64Audio, index: index });
    // Ordenar la cola por índice para mantener el orden correcto
    audioQueue.sort((a, b) => a.index - b.index);
    playNextAudioClip();
}

function playNextAudioClip() {
    if (audioQueue.length > 0 && audioQueue[0].index === currentAudioIndex) {
        const audioData = audioQueue.shift();
        currentAudioIndex++;

        let audioContainer = document.getElementById('audioPlayerContainer');
        audioContainer.innerHTML = ''; // Limpia el contenedor

        let audioSrc = `data:audio/wav;base64,${audioData.audio}`;
        let audioPlayer = document.createElement('audio');
        audioPlayer.src = audioSrc;
        audioPlayer.controls = true;
        audioPlayer.autoplay = true;

        audioContainer.appendChild(audioPlayer);

        audioPlayer.onended = playNextAudioClip;
    }
}


function createAudioPlayer(base64Audio) {
    let audioContainer = document.getElementById('audioPlayerContainer');
    let audioSrc = `data:audio/wav;base64,${base64Audio}`;
    let audioPlayer = document.createElement('audio');
    audioPlayer.src = audioSrc;
    audioPlayer.controls = true;
    audioPlayer.autoplay = true;
    audioContainer.innerHTML = '';
    audioContainer.appendChild(audioPlayer);
}




</script>

Overwriting miniJuegoClienteBuffer.html


### Cliente javascript que se conecta de manera local

In [14]:
%%writefile miniJuegoCliente.html

<head>
  <style>


    body {
        font-family: Arial, sans-serif; /* Mejora la tipografía general */
    }


    #system_prompt {
        height: 150px;
    }


    #inicioForm {
        max-width: 1000px; /* Limita el ancho del formulario */
        margin: 20px auto; /* Centra el formulario */
        padding: 20px;
        box-shadow: 0 0 10px rgba(0, 0, 0, 0.1); /* Añade un sombreado ligero */
    }

    #inicioForm div {
        margin-bottom: 15px; /* Añade más espacio entre los campos */
    }

    #inicioForm label {
        font-weight: bold; /* Hace que las etiquetas sean más notables */
        display: block; /* Asegura que la etiqueta esté encima del input */
        margin-bottom: 5px; /* Espacio entre la etiqueta y el campo */
    }

    #inicioForm select, #inicioForm textarea, #inicioForm button {
        width: 100%; /* Aprovecha todo el ancho disponible */
        padding: 8px; /* Añade un relleno para mayor comodidad */
        margin-top: 4px; /* Espacio mínimo superior para separación */
    }

    #inicioForm select {
        cursor: pointer; /* Indica que es un elemento interactivo */
        font-size: 16px; /* Aumenta el tamaño del texto */
    }

    #inicioForm textarea {
        resize: vertical; /* Permite al usuario ajustar la altura verticalmente */
    }

    #inicioForm button {
        background-color: #007bff; /* Color de fondo */
        color: white; /* Color del texto */
        border: none; /* Elimina el borde */
        padding: 10px 15px; /* Añade relleno */
        font-size: 18px; /* Aumenta el tamaño del texto */
        cursor: pointer; /* Indica que es un elemento interactivo */
        border-radius: 5px; /* Bordes redondeados */
    }

    #inicioForm button:hover {
        background-color: #0056b3; /* Oscurece el botón al pasar el mouse */
    }

#audioPlayerContainer {
    /* Añade estilos específicos si planeas insertar un reproductor de audio */
    margin-bottom: 20px; /* Espacio antes del botón de grabación */
}

#recordButton {
    background-color: #f44336; /* Color rojo para la grabación */
    color: white;
    border: none;
    padding: 10px 0;
    font-size: 18px;
    border-radius: 5px;
    cursor: pointer;
}

#recordButton:hover {
    background-color: #d32f2f; /* Oscurece el botón al pasar el mouse */
}

/* Estilos para el área de texto y botón de envío */
/* Contenedor del área de texto y el botón */
div#textButtonContainer {
    display: flex; /* Establece el contenedor para usar flexbox */
    justify-content: space-between; /* Espacia los elementos uniformemente */
    align-items: center; /* Alinea los elementos verticalmente en el centro */
}

/* Área de texto */
#textInput {
    flex-grow: 1; /* Permite que el área de texto crezca para ocupar el espacio disponible */
    margin-right: 10px; /* Añade un margen a la derecha para separarlo del botón */
    border: 1px solid #ccc; /* Establece un borde sutil */
    border-radius: 5px; /* Bordes redondeados */
    padding: 8px; /* Añade padding interno */
}

/* Botón */
#sendTextButton {
    padding: 8px 15px; /* Ajusta el padding para dimensionar el botón */
    background-color: #4CAF50; /* Color de fondo */
    color: white; /* Color del texto */
    border: none; /* Elimina el borde */
    border-radius: 5px; /* Bordes redondeados */
    cursor: pointer; /* Cambia el cursor a mano al pasar sobre el botón */
}

#sendTextButton:hover {
    background-color: #388E3C; /* Oscurece el botón al pasar el mouse */
}


#responseText {
    height: 250px;
    margin-top: 20px;
    border: 1px solid #ddd;
    padding: 10px;
    overflow-y: auto; /* Asegura el desplazamiento vertical */
    background-color: #f9f9f9; /* Fondo claro para resaltar el área */
}



</style>
</head>

<script>

document.addEventListener('DOMContentLoaded', function() {
    window.scrollTo(0, 0); // Asegura que la página comience en la parte superior
    document.getElementById('ejercicios').focus(); // Luego establece el foco en el selector
});



// Objeto para mapear los ejercicios a sus strings correspondientes
const ejerciciosStrings = {
    "guessing_game1": {
        systemPrompt: `This is a conversational game in which you have to think in this famous character: #personaje, and the user have to guess this character. You should aswer the user questions about the character. Be concise but give some clues. NEVER say the name of the character until the end. If the user guesses the character then you will say: "Congratulations, you got it right, the character was #personaje". If user give up you will say: "what a shame!, the character was #personaje
Game success example
assistant: I'm thinking of a famous fictional character, guess which one it is.
user: is it real or fictional?
assistant: it is fictional
user: Is he #personaje?
assistant: Congratulations, you were right, it was #personaje.
Game give up example
assistant: I'm thinking of a famous fictional character, guess which one it is.
user: is it real or fictional?
assistant: it is fictional
user: Is he Benito Perez?
assistant: No, it has nothing to do with.
user: I give up. Who is it?
assistant: what a shame!, the character was #personaje
`,
        saludo: "I'm thinking of a famous fictional character, guess which one it is."
    },
    "guessing_game2": {
        systemPrompt: "You are #personaje, the fictional character. You have to take on the personality of that character and engage in conversations about your events and experiences.",
        saludo: "I'm #personaje, the fictional character. Ask me anything you want to know about me."
    },    
    "guessing_game3": {
        systemPrompt: "This is a conversational game in which you have to guess a famous character. You should make questions to the user in order to guess the character that the user have choosen.",
        saludo: "Do you want to play? I will guess your choosen character by asking about it."
    },
    "yes_no_game": {
        systemPrompt: `IMPORTANT: You only can answer "yes" or "no" (nothing more!).
This is a conversational game between you and a the user. The game consists that only at the beggininig you tell the user only a piece of the context and the user having to guess the "key point" of the context from "yes or no questions". The User could ask anything about the story (context) to guess the "key point" of the context but Assistant could only answer "Yes" or "not". If the user asks a question that does not lend itself or cannot be answered with a "yes" or "no" such as "What is the man's name?" then Assistant will respond: "Only "yes or no" questions. When the user guesses the key point of the story you will say: "Congratulations, you have guessed the key to the story.
Game context: A man named Edgar is the lighthouse keeper of Águilas for 30 years. He always turns on the lighthouse at dusk and shortly after sleeps in a small room next to its large lamp. On Edgar's birthday, at dusk after lighting the lighthouse, he decided to go to dinner with an old friend to celebrate his birthday. During dinner he drank more than necessary and they both got drunk. Afther the dinner Edgar accompanied his friend to her house and then went to his house, the lighthouse, where he sleeps every night. Upon entering the lighthouse and going up to his room, due to his drunkenness and the fact that he was very sleepy, he decided to turn off the light (which was actually the light from the main lamp of the lighthouse) to sleep off the drunkenness and did it without realizing or knowing it was dangerous. During the early hours of the morning, a cruise ship full of passengers crashed into the cliff that the lighthouse protected because, when it was turned off, neither the lookout, nor the captain, nor the rest of the crew nor the passengers could see that they were heading against the cliff. An hour later Edgar wakes up, it hasn't dawned yet but you can hear sirens and a lot of noise from the rescuers who are trying to rescue the shipwrecked. Edgar turns on the lamp to illuminate the scene where hundreds of dead shipwrecked people continually crash against the cliff due to the waves. Faced with this heartbreaking reality and his feeling of guilt, Edgar decides to commit suicide by jumping from the top of the lighthouse. The key point of the story that the user must find out is: "Edgar commits suicide because he was the LIGHTHOUSE keeper." or similar but always emphasizing that he was the lighthouse keeper.
IMPORTANT: You only can answer "yes" or "no" or "Only yes or no questions"
Examples of correct answers:
user: What is Edgar's job?
Assistant: Only yes or no questions.
user: Is Edgar a man?
Assistant: yes.`,
        saludo: `This is I can show about the hidden story: Edgar was dazed and comes to his room, turns off the light and lies down on his bed. He wakes up a few hours later, turns on the light, looks out the window and is so horrified that he ends up jumping out of the window and committing suicide.
Guess what happened.
IMPORTANT: From now on I can only answer you "yes" or "no" and nothing more.`
    },
    "English_teacher": {
        systemPrompt: "You are an english teacher. You will have simple dialogues with the student in your charge. you will only have concise conversations with short sentences so that the student is encouraged to converse.",
        saludo: "Good Morning. What is your name?"
    }
    // Añade más ejercicios según sea necesario
};
</script>


<form id="inicioForm">
    <div>
        <label for="system_prompt">System Prompt:</label><br>
        <textarea id="system_prompt" name="system_prompt" rows="10" cols="100"></textarea>
    </div>
    <div>
        <label for="saludo">Saludo:</label><br>
        <textarea id="saludo" name="saludo" rows="4" cols="100"></textarea>
    </div>
    <div>
        <label for="ejercicios">Juegos:</label><br>
        <select id="ejercicios" name="ejercicios">
            <option value="">Selecciona un ejercicio</option>
            <option value="English_teacher">English teacher</option>
            <option value="guessing_game1">Guessing game 1</option>
            <option value="guessing_game2">Guessing game 2</option>
            <option value="guessing_game3">Guessing game 3</option>
            <option value="yes_no_game">yes no game</option>
        </select>
    </div>
    <button type="submit">EMPEZAR!</button>

</form>


<div style="margin-bottom: 20px; display: flex; align-items: center;"> 
    <button id="downloadButton" style="background-color: #4CAF50; /* Color de fondo */
                                       color: white; /* Color del texto */
                                       padding: 15px 32px; /* Padding alrededor del texto */
                                       text-align: center; /* Alinea el texto al centro */
                                       text-decoration: none; /* Elimina la decoración del texto */
                                       display: inline-block; /* Hace que el botón sea un bloque en línea */
                                       font-size: 16px; /* Tamaño del texto */
                                       margin: 4px 2px; /* Margen alrededor del botón */
                                       cursor: pointer; /* Cambia el cursor a un puntero */
                                       border: none; /* Elimina el borde */
                                       border-radius: 8px; /* Redondea las esquinas del botón */
    ">Descargar Conversación</button>
    <div id="audioPlayerAllContainer"></div>
</div>


<script>
    document.getElementById('downloadButton').addEventListener('click', function() {
        obtenerYReproducirAll(); // Llama a la función en vez de redirigir
    });
</script>


<script>

document.getElementById('ejercicios').addEventListener('change', function() {
    var selectedKey = this.value; // La clave seleccionada del objeto
    if (selectedKey) {
        // Actualiza los textareas con los valores correspondientes
        document.getElementById('system_prompt').value = ejerciciosStrings[selectedKey].systemPrompt;
        document.getElementById('saludo').value = ejerciciosStrings[selectedKey].saludo;
    }
});


document.getElementById('inicioForm').addEventListener('submit', function(e) {
    // Prevenir el comportamiento predeterminado del formulario
    e.preventDefault();

    // Obtener los valores de los campos del formulario
    const system_prompt = document.getElementById('system_prompt').value;
    const saludo = document.getElementById('saludo').value;

    document.getElementById('responseText').innerText = ""
    //obtenerYReproducirAudio(saludo)
    //updateResponseText(saludo + "\n")

    // Crear el cuerpo de la solicitud
    const data = { system_prompt, saludo };

    // Realizar la llamada al servicio Flask
    fetch('http://localhost:5500/inicio', {
        method: 'POST',
        headers: {
            'Content-Type': 'application/json',
        },
        body: JSON.stringify(data),
    })
    .then(response => response.json())
    .then(data => {
        let saludo = data.message;
        //document.getElementById('saludo').value = saludo;
        obtenerYReproducirAudio(saludo)
        updateResponseText(saludo + "\n")
        alert('Nuevo Juego iniciado');
    })
    .catch((error) => {
        console.error('Error:', error);
        alert('Error al enviar los strings');
    });
});
</script>


<div id="audioPlayerContainer"></div>
<button id="recordButton" style="width: 100%; height: 50px;">Pulsa para grabar/detener</button>


<!-- Estilos para textarea y botón de envío -->
<div id="textButtonContainer" style="margin-top: 10px;">
    <textarea id="textInput" placeholder="Escribe tu texto aquí" rows="4"></textarea>
    <button id="sendTextButton">Enviar Texto</button>
</div>



<div id="responseText" style="height: 250px; margin-top: 20px; border: 1px solid #ddd; padding: 10px; overflow-y:auto;"></div>
<script>
let recordButton = document.getElementById("recordButton");
let chunks = [];
let mediaRecorder;
let isRecording = false;

navigator.mediaDevices.getUserMedia({ audio: true })
.then(stream => {
    mediaRecorder = new MediaRecorder(stream);
    mediaRecorder.ondataavailable = event => {
        chunks.push(event.data);
    };
    mediaRecorder.onstop = () => {
        console.log('mediaRecorder Detenido!!');
        let blob = new Blob(chunks, { 'type': 'audio/ogg; codecs=opus' });
        enviarAudioAlServidor(blob);
        chunks = [];
    };
});

recordButton.onclick = () => {
    if (!isRecording) {
        mediaRecorder.start();
        isRecording = true;
        recordButton.textContent = 'Grabando...';
    } else {
        mediaRecorder.stop();
        isRecording = false;
        recordButton.textContent = 'Pulsa para grabar/detener';
    }
};


function enviarAudioAlServidor(blob) {
    let formData = new FormData();
    formData.append('file', blob, 'grabacion.ogg');

    fetch('http://localhost:5500/transcribe', {
        method: 'POST',
        body: formData,
    })
    .then(response => response.json())
    .then(data => {
        updateResponseText("\nyo: " + data.entrada + "\n\n" + "\n***********************************\n" + "respuesta: ");
        let obtenerPartes = () => {
            fetch('http://localhost:5500/get_next_part') // Cambia la URL según tu configuración
            .then(response => response.json())
            .then(partData => {
                if (partData.output !== "") { // Mientras la parte de la respuesta no esté vacía
                    trozo = partData.output;
                    updateResponseText(trozo);
                    obtenerYReproducirAudio(trozo);
                    obtenerPartes(); // Vuelve a llamar a obtenerPartes recursivamente
                } 
            })
            .catch(error => {
                console.error('Error al obtener la siguiente parte:', error);
            });
        };

        obtenerPartes(); // Comienza a obtener las partes de la respuesta
    })
    .catch(error => {
        console.error('Error al enviar el audio:', error);
    });
}


// Funcionalidad para enviar texto al servidor y limpiar el textarea
document.getElementById("sendTextButton").onclick = () => {
    let textInput = document.getElementById("textInput");
    let texto = textInput.value;
    if (texto) {
        enviarTextoAlServidor(texto);
        textInput.value = ''; // Limpiar el textarea después de enviar
    }
};

function enviarTextoAlServidor(texto) {
    fetch('http://localhost:5500/texto', { // Asegúrate de usar la URL correcta
        method: 'POST',
        headers: {
            'Content-Type': 'application/json'
        },
        body: JSON.stringify({ texto: texto })
    })
    .then(response => response.json())
    .then(data => {
        updateResponseText("\nyo: " + data.entrada + "\n\n" + "\n***********************************\n" + "respuesta: ");
        let obtenerPartes = () => {
            fetch('http://localhost:5500/get_next_part') // Asegúrate de usar la URL correcta
            .then(response => response.json())
            .then(partData => {
                if (partData.output !== "") { // Mientras la parte de la respuesta no esté vacía
                    trozo = partData.output;
                    obtenerYReproducirAudio(trozo);
                    obtenerPartes(); // Vuelve a llamar a obtenerPartes recursivamente
                } 
            })
            .catch(error => {
                console.error('Error al obtener la siguiente parte:', error);
            });
        };

        obtenerPartes(); // Comienza a obtener las partes de la respuesta
    })
    .catch(error => {
        console.error('Error al enviar el texto:', error);
    });
}


function updateResponseText(text) {
    // document.getElementById('responseText').innerText += text;
    textarea = document.getElementById('responseText')
    textarea.innerText += text;
    textarea.scrollTop = textarea.scrollHeight;
}



function obtenerYReproducirAll() {
    fetch('http://localhost:5500/all_conversation', {
        method: 'GET',
    })
    .then(response => response.json())
    .then(data => {
        if (data.audio_base64) {
            createAudioAllPlayer(data.audio_base64);
        }
    })
    .catch(error => {
        console.error('Error al obtener el audio:', error);
    });
}


function createAudioAllPlayer(base64Audio) {
    let audioContainer = document.getElementById('audioPlayerAllContainer');
    let audioSrc = `data:audio/wav;base64,${base64Audio}`;
    let audioPlayer = document.createElement('audio');
    audioPlayer.src = audioSrc;
    audioPlayer.controls = true;
    audioPlayer.autoplay = true;
    audioContainer.innerHTML = '';
    audioContainer.appendChild(audioPlayer);
}




function obtenerYReproducirAudio(texto) {
    fetch('http://localhost:5500/audio', {

        method: 'POST',
        headers: {
            'Content-Type': 'application/json'
        },
        body: JSON.stringify({ texto: texto })
    })
    .then(response => response.json())
    .then(data => {
        if (data.audio_base64) {
            addAudioClipToQueue(data.audio_base64);
        }
    })
    .catch(error => {
        console.error('Error al obtener el audio:', error);
    });
}



let audioClips = []; // Cola para almacenar los clips de audio base64
let isPlaying = false; // Indicador de si hay un clip de audio actualmente en reproducción

function playNextAudioClip() {
    if (isPlaying || audioClips.length === 0) {
        return; // Si ya se está reproduciendo audio o no hay clips en la cola, no hacer nada
    }

    // Marcar que se iniciará la reproducción de un clip
    isPlaying = true;

    // Obtener el siguiente clip de audio de la cola
    const base64Audio = audioClips.shift(); // Esto elimina el primer elemento de la cola y lo retorna

    let audioContainer = document.getElementById('audioPlayerContainer');
    if (!audioContainer) return; // Asegurarse de que el contenedor existe

    let audioSrc = `data:audio/wav;base64,${base64Audio}`;
    let audioPlayer = document.createElement('audio');
    audioPlayer.src = audioSrc;
    audioPlayer.controls = true;
    audioPlayer.autoplay = true;

    // Limpiar el contenedor de audio solo si es necesario
    audioContainer.innerHTML = '';
    audioContainer.appendChild(audioPlayer);

    audioPlayer.onended = () => {
        isPlaying = false; // Marcar que la reproducción del clip actual ha terminado
        playNextAudioClip(); // Intentar reproducir el siguiente clip en la cola
    };
}

// Función para añadir clips de audio a la cola y tratar de reproducir
function addAudioClipToQueue(base64Audio) {
    audioClips.push(base64Audio); // Añadir el nuevo clip de audio a la cola
    playNextAudioClip(); // Intentar reproducir el siguiente clip
}

function createAudioPlayer(base64Audio) {
    let audioContainer = document.getElementById('audioPlayerContainer');
    let audioSrc = `data:audio/wav;base64,${base64Audio}`;
    let audioPlayer = document.createElement('audio');
    audioPlayer.src = audioSrc;
    audioPlayer.controls = true;
    audioPlayer.autoplay = true;
    audioContainer.innerHTML = '';
    audioContainer.appendChild(audioPlayer);
}




</script>

Overwriting miniJuegoCliente.html


### cliente javascript que se conecta de manera remota

Eres Edward, el CEO de una consultora llamada mAgIc, dedicada a buscar soluciones de inteligencia artificial para empresas. Estás entrevistando a un candidato para trabajar en la empresa, te interesan los conocimientos de programación, matemáticas, ciencia de datos, IA y especialmente PNL. Haz preguntas sobre todo esto e indaga. Si el candidato dice algo incorrecto házselo saber. Después de 4 preguntas, si la mayoría fueron buenas respuestas, debes decir "¡¡¡FELICIDADES!!! ¡¡¡ESTÁS EN NUESTRO EQUIPO!!!" Si hubo una mala respuesta, entonces debes decir "La entrevista ha terminado, gracias por venir" IMPORTANTE: si el entrevistado es grosero, entonces debes decir "vete, cierra la puerta cuando salgas".

Buenos días, bienvenidos a MAgIc, tomen asiento, díganme su nombre.

In [38]:
%%writefile miniJuegoClienteRemoto.html

# @title DESCARGA DE FICHERO COMPLETO!!!! JUEGO CONVERSACIONAL (NO OLVIDES DAR AL PLAY PRIMERO)

%%html

<head>
  <style>


    body {
        font-family: Arial, sans-serif; /* Mejora la tipografía general */
    }


    #system_prompt {
        height: 150px;
    }




    #inicioForm {
        max-width: 1000px; /* Limita el ancho del formulario */
        margin: 20px auto; /* Centra el formulario */
        padding: 20px;
        box-shadow: 0 0 10px rgba(0, 0, 0, 0.1); /* Añade un sombreado ligero */
    }

    #inicioForm div {
        margin-bottom: 15px; /* Añade más espacio entre los campos */
    }

    #inicioForm label {
        font-weight: bold; /* Hace que las etiquetas sean más notables */
        display: block; /* Asegura que la etiqueta esté encima del input */
        margin-bottom: 5px; /* Espacio entre la etiqueta y el campo */
    }

    #inicioForm select, #inicioForm textarea, #inicioForm button {
        width: 100%; /* Aprovecha todo el ancho disponible */
        padding: 8px; /* Añade un relleno para mayor comodidad */
        margin-top: 4px; /* Espacio mínimo superior para separación */
    }

    #inicioForm select {
        cursor: pointer; /* Indica que es un elemento interactivo */
        font-size: 16px; /* Aumenta el tamaño del texto */
    }

    #inicioForm textarea {
        resize: vertical; /* Permite al usuario ajustar la altura verticalmente */
    }

    #inicioForm button {
        background-color: #007bff; /* Color de fondo */
        color: white; /* Color del texto */
        border: none; /* Elimina el borde */
        padding: 10px 15px; /* Añade relleno */
        font-size: 18px; /* Aumenta el tamaño del texto */
        cursor: pointer; /* Indica que es un elemento interactivo */
        border-radius: 5px; /* Bordes redondeados */
    }

    #inicioForm button:hover {
        background-color: #0056b3; /* Oscurece el botón al pasar el mouse */
    }

#audioPlayerContainer {
    /* Añade estilos específicos si planeas insertar un reproductor de audio */
    margin-bottom: 20px; /* Espacio antes del botón de grabación */
}

#recordButton {
    background-color: #f44336; /* Color rojo para la grabación */
    color: white;
    border: none;
    padding: 10px 0;
    font-size: 18px;
    border-radius: 5px;
    cursor: pointer;
}

#recordButton:hover {
    background-color: #d32f2f; /* Oscurece el botón al pasar el mouse */
}

/* Estilos para el área de texto y botón de envío */
/* Contenedor del área de texto y el botón */
div#textButtonContainer {
    display: flex; /* Establece el contenedor para usar flexbox */
    justify-content: space-between; /* Espacia los elementos uniformemente */
    align-items: center; /* Alinea los elementos verticalmente en el centro */
}

/* Área de texto */
#textInput {
    flex-grow: 1; /* Permite que el área de texto crezca para ocupar el espacio disponible */
    margin-right: 10px; /* Añade un margen a la derecha para separarlo del botón */
    border: 1px solid #ccc; /* Establece un borde sutil */
    border-radius: 5px; /* Bordes redondeados */
    padding: 8px; /* Añade padding interno */
}

/* Botón */
#sendTextButton {
    padding: 8px 15px; /* Ajusta el padding para dimensionar el botón */
    background-color: #4CAF50; /* Color de fondo */
    color: white; /* Color del texto */
    border: none; /* Elimina el borde */
    border-radius: 5px; /* Bordes redondeados */
    cursor: pointer; /* Cambia el cursor a mano al pasar sobre el botón */
}

#sendTextButton:hover {
    background-color: #388E3C; /* Oscurece el botón al pasar el mouse */
}


#responseText {
    height: 250px;
    margin-top: 20px;
    border: 1px solid #ddd;
    padding: 10px;
    overflow-y: auto; /* Asegura el desplazamiento vertical */
    background-color: #f9f9f9; /* Fondo claro para resaltar el área */
}



</style>
</head>

<script>

document.addEventListener('DOMContentLoaded', function() {
    window.scrollTo(0, 0); // Asegura que la página comience en la parte superior
    document.getElementById('ejercicios').focus(); // Luego establece el foco en el selector
});



// Objeto para mapear los ejercicios a sus strings correspondientes
const ejerciciosStrings = {
    "guessing_game1": {
        systemPrompt: "This is a conversational game in which you have to think in a famous character and the user have to guess this character. You should aswer the user questions about the character.",
        saludo: "Do you want to play?"
    },
    "guessing_game2": {
        systemPrompt: "This is a conversational game in which you have to guess a famous character. You should make questions to the user in order to guess the character that the user have choosen.",
        saludo: "Do you want to play? I will guess your choosen character by asking about it."
    },
    "yes_no_game": {
        systemPrompt: `IMPORTANT: You only can answer "yes" or "no" (nothing more!).
This is a conversational game between you and a the user. The game consists that only at the beggininig you tell the user only a piece of the context and the user having to guess the "key point" of the context from "yes or no questions". The User could ask anything about the story (context) to guess the "key point" of the context but Assistant could only answer "Yes" or "not". If the user asks a question that does not lend itself or cannot be answered with a "yes" or "no" such as "What is the man's name?" then Assistant will respond: "Only "yes or no" questions. When the user guesses the key point of the story you will say: "Congratulations, you have guessed the key to the story.
Game context: A man named Edgar is the lighthouse keeper of Águilas for 30 years. He always turns on the lighthouse at dusk and shortly after sleeps in a small room next to its large lamp. On Edgar's birthday, at dusk after lighting the lighthouse, he decided to go to dinner with an old friend to celebrate his birthday. During dinner he drank more than necessary and they both got drunk. Afther the dinner Edgar accompanied his friend to her house and then went to his house, the lighthouse, where he sleeps every night. Upon entering the lighthouse and going up to his room, due to his drunkenness and the fact that he was very sleepy, he decided to turn off the light (which was actually the light from the main lamp of the lighthouse) to sleep off the drunkenness and did it without realizing or knowing it was dangerous. During the early hours of the morning, a cruise ship full of passengers crashed into the cliff that the lighthouse protected because, when it was turned off, neither the lookout, nor the captain, nor the rest of the crew nor the passengers could see that they were heading against the cliff. An hour later Edgar wakes up, it hasn't dawned yet but you can hear sirens and a lot of noise from the rescuers who are trying to rescue the shipwrecked. Edgar turns on the lamp to illuminate the scene where hundreds of dead shipwrecked people continually crash against the cliff due to the waves. Faced with this heartbreaking reality and his feeling of guilt, Edgar decides to commit suicide by jumping from the top of the lighthouse. The key point of the story that the user must find out is: "Edgar commits suicide because he was the LIGHTHOUSE keeper." or similar but always emphasizing that he was the lighthouse keeper.
IMPORTANT: You only can answer "yes" or "no" or "Only yes or no questions"
Examples of correct answers:
user: What is Edgar's job?
Assistant: Only yes or no questions.
user: Is Edgar a man?
Assistant: yes.`,
        saludo: `This is I can show about the hidden story: Edgar was dazed and comes to his room, turns off the light and lies down on his bed. He wakes up a few hours later, turns on the light, looks out the window and is so horrified that he ends up jumping out of the window and committing suicide.
Guess what happened.
IMPORTANT: From now on I can only answer you "yes" or "no" and nothing more.`
    },
    "English_teacher": {
        systemPrompt: "You are an english teacher. You will have simple dialogues with the student in your charge. You will correct the wrong things he says. At first you will ask your student's name and then you will ask them to choose between having a conversation or practicing grammar.",
        saludo: "Good Morning. What is your name?"
    }
    // Añade más ejercicios según sea necesario
};
</script>


<form id="inicioForm">
    <div>
        <label for="system_prompt">System Prompt:</label><br>
        <textarea id="system_prompt" name="system_prompt" rows="10" cols="100"></textarea>
    </div>
    <div>
        <label for="saludo">Saludo:</label><br>
        <textarea id="saludo" name="saludo" rows="4" cols="100"></textarea>
    </div>
    <div>
        <label for="ejercicios">Juegos:</label><br>
        <select id="ejercicios" name="ejercicios">
            <option value="">Selecciona un ejercicio</option>
            <option value="English_teacher">English teacher</option>
            <option value="guessing_game1">Guessing game 1</option>
            <option value="guessing_game2">Guessing game 2</option>
            <option value="yes_no_game">yes no game</option>
        </select>
    </div>
    <button type="submit">EMPEZAR!</button>

</form>


<div style="margin-bottom: 20px; display: flex; align-items: center;"> <!-- Uso de Flexbox para alinear elementos horizontalmente y centrarlos verticalmente -->
    <button id="downloadButton" style="background-color: #4CAF50; /* Color de fondo */
                                       color: white; /* Color del texto */
                                       padding: 15px 32px; /* Padding alrededor del texto */
                                       text-align: center; /* Alinea el texto al centro */
                                       text-decoration: none; /* Elimina la decoración del texto */
                                       display: inline-block; /* Hace que el botón sea un bloque en línea */
                                       font-size: 16px; /* Tamaño del texto */
                                       margin: 4px 2px; /* Margen alrededor del botón */
                                       cursor: pointer; /* Cambia el cursor a un puntero */
                                       border: none; /* Elimina el borde */
                                       border-radius: 8px; /* Redondea las esquinas del botón */
    ">Descargar Conversación</button>
    <div id="audioPlayerAllContainer"></div>
</div>


<script>
    document.getElementById('downloadButton').addEventListener('click', function() {
        obtenerYReproducirAll(); // Llama a la función en vez de redirigir
    });
</script>


<script>

document.getElementById('ejercicios').addEventListener('change', function() {
    var selectedKey = this.value; // La clave seleccionada del objeto
    if (selectedKey) {
        // Actualiza los textareas con los valores correspondientes
        document.getElementById('system_prompt').value = ejerciciosStrings[selectedKey].systemPrompt;
        document.getElementById('saludo').value = ejerciciosStrings[selectedKey].saludo;
    }
});


document.getElementById('inicioForm').addEventListener('submit', function(e) {
    // Prevenir el comportamiento predeterminado del formulario
    e.preventDefault();

    // Obtener los valores de los campos del formulario
    const system_prompt = document.getElementById('system_prompt').value;
    const saludo = document.getElementById('saludo').value;

    document.getElementById('responseText').innerText = ""
    obtenerYReproducirAudio(saludo)
    updateResponseText(saludo + "\n")

    // Crear el cuerpo de la solicitud
    const data = { system_prompt, saludo };

    // Realizar la llamada al servicio Flask
    fetch('http://ia.javiergimenez.es:5500/inicio', {
        method: 'POST',
        headers: {
            'Content-Type': 'application/json',
        },
        body: JSON.stringify(data),
    })
    .then(response => response.json())
    .then(data => {
        console.log('Success:', data);
        alert('Nuevo Juego iniciado');
    })
    .catch((error) => {
        console.error('Error:', error);
        alert('Error al enviar los strings');
    });
});
</script>


<div id="audioPlayerContainer"></div>
<button id="recordButton" style="width: 100%; height: 50px;">Pulsa para grabar/detener</button>


<!-- Estilos para textarea y botón de envío -->
<div id="textButtonContainer" style="margin-top: 10px;">
    <textarea id="textInput" placeholder="Escribe tu texto aquí" rows="4"></textarea>
    <button id="sendTextButton">Enviar Texto</button>
</div>



<div id="responseText" style="height: 250px; margin-top: 20px; border: 1px solid #ddd; padding: 10px; overflow-y:auto;"></div>
<script>
let recordButton = document.getElementById("recordButton");
let chunks = [];
let mediaRecorder;
let isRecording = false;

navigator.mediaDevices.getUserMedia({ audio: true })
.then(stream => {
    mediaRecorder = new MediaRecorder(stream);
    mediaRecorder.ondataavailable = event => {
        chunks.push(event.data);
    };
    mediaRecorder.onstop = () => {
        console.log('mediaRecorder Detenido!!');
        let blob = new Blob(chunks, { 'type': 'audio/ogg; codecs=opus' });
        enviarAudioAlServidor(blob);
        chunks = [];
    };
});

recordButton.onclick = () => {
    if (!isRecording) {
        mediaRecorder.start();
        isRecording = true;
        recordButton.textContent = 'Grabando...';
    } else {
        mediaRecorder.stop();
        isRecording = false;
        recordButton.textContent = 'Pulsa para grabar/detener';
    }
};

function enviarAudioAlServidor(blob) {
    let formData = new FormData();
    formData.append('file', blob, 'grabacion.ogg');

    //fetch('http://localhost:5500/transcribe', {
    fetch('http://ia.javiergimenez.es:5500/transcribe', {

        method: 'POST',
        body: formData,
    })
    .then(response => response.json())
    .then(data => {
        updateResponseText("yo: " + data.entrada + "\n\n" + (data.entrada_traducida ? "yo (traducción): " + data.entrada_traducida + "\n" : "\n") + "\n***********************************\n" + "respuesta: " + data.respuesta + "\n\n" + (data.respuesta_traducida ? "Respuesta (traducida): " + data.respuesta_traducida + "\n" : ""));
        if (data.respuesta_traducida) {
            obtenerYReproducirAudio(data.respuesta_traducida);
        }
        else if (data.respuesta) {
            obtenerYReproducirAudio(data.respuesta);
        }
    })
    .catch(error => {
        console.error('Error al enviar el audio:', error);
    });
}

// Funcionalidad para enviar texto al servidor y limpiar el textarea
document.getElementById("sendTextButton").onclick = () => {
    let textInput = document.getElementById("textInput");
    let texto = textInput.value;
    if (texto) {
        enviarTextoAlServidor(texto);
        textInput.value = ''; // Limpiar el textarea después de enviar
    }
};

function enviarTextoAlServidor(texto) {
    //fetch('http://localhost:5500/texto', {
    fetch('http://ia.javiergimenez.es:5500/texto', {

        method: 'POST',
        headers: {
            'Content-Type': 'application/json'
        },
        body: JSON.stringify({ texto: texto })
    })
    .then(response => response.json())
    .then(data => {
        updateResponseText("yo: " + data.entrada + "\n\n" + (data.entrada_traducida ? "yo (traducción): " + data.entrada_traducida + "\n" : "\n") + "\n***********************************\n" + "respuesta: " + data.respuesta + "\n\n" + (data.respuesta_traducida ? "Respuesta (traducida): " + data.respuesta_traducida + "\n" : ""));
        if (data.respuesta_traducida) {
            obtenerYReproducirAudio(data.respuesta_traducida);
        }
        else if (data.respuesta) {
            obtenerYReproducirAudio(data.respuesta);
        }
    })
    .catch(error => {
        console.error('Error al enviar el texto:', error);
    });
}

function updateResponseText(text) {
    // document.getElementById('responseText').innerText += text;
    textarea = document.getElementById('responseText')
    textarea.innerText += text;
    textarea.scrollTop = textarea.scrollHeight;
}



function obtenerYReproducirAll() {
    fetch('http://ia.javiergimenez.es:5500/all_conversation', {
        method: 'GET',
    })
    .then(response => response.json())
    .then(data => {
        if (data.audio_base64) {
            createAudioAllPlayer(data.audio_base64);
        }
    })
    .catch(error => {
        console.error('Error al obtener el audio:', error);
    });
}


function createAudioAllPlayer(base64Audio) {
    let audioContainer = document.getElementById('audioPlayerAllContainer');
    let audioSrc = `data:audio/wav;base64,${base64Audio}`;
    let audioPlayer = document.createElement('audio');
    audioPlayer.src = audioSrc;
    audioPlayer.controls = true;
    audioPlayer.autoplay = true;
    audioContainer.innerHTML = '';
    audioContainer.appendChild(audioPlayer);
}



function obtenerYReproducirAudio(texto) {
    fetch('http://ia.javiergimenez.es:5500/audio', {

        method: 'POST',
        headers: {
            'Content-Type': 'application/json'
        },
        body: JSON.stringify({ texto: texto })
    })
    .then(response => response.json())
    .then(data => {
        if (data.audio_base64) {
            createAudioPlayer(data.audio_base64);
        }
    })
    .catch(error => {
        console.error('Error al obtener el audio:', error);
    });
}

function createAudioPlayer(base64Audio) {
    let audioContainer = document.getElementById('audioPlayerContainer');
    let audioSrc = `data:audio/wav;base64,${base64Audio}`;
    let audioPlayer = document.createElement('audio');
    audioPlayer.src = audioSrc;
    audioPlayer.controls = true;
    audioPlayer.autoplay = true;
    audioContainer.innerHTML = '';
    audioContainer.appendChild(audioPlayer);
}


</script>

Overwriting miniJuegoClienteRemoto.html


# Comprobaciones

In [1]:
!pm2 list

┌────┬────────────────────┬──────────┬──────┬───────────┬──────────┬──────────┐
│ id │ name               │ mode     │ ↺    │ status    │ cpu      │ memory   │
├────┼────────────────────┼──────────┼──────┼───────────┼──────────┼──────────┤
│ 0  │ server_conversaci… │ fork     │ 0    │ online    │ 0%       │ 13.6gb   │
└────┴────────────────────┴──────────┴──────┴───────────┴──────────┴──────────┘


In [11]:
!lsof -i :5500

COMMAND   PID   USER   FD   TYPE DEVICE SIZE/OFF NODE NAME
python  27729 javier   19u  IPv4 506966      0t0  TCP *:5500 (LISTEN)
python  27729 javier   23u  IPv4 493826      0t0  TCP 172.25.241.247:5500->javierAIMachine:54930 (ESTABLISHED)


Verificamos con netcat que está el puerto habierto

In [23]:
!nc -vz localhost 5500

Connection to localhost (127.0.0.1) 5500 port [tcp/*] succeeded!


Verificamos con Curl que el servicio POST funciona (le enviamos un fichero y nos devuelve el texto)

En el comando curl, la arroba (@) se utiliza para indicar que lo que sigue es el nombre de un archivo del cual curl debe leer los datos para incluirlos en el cuerpo de la petición. Cuando usas -F (o --form) para construir una petición multipart/form-data, la arroba permite especificar el archivo a enviar en el campo correspondiente del formulario.

In [ ]:
# !curl -F "file=@grabacion.ogg"  http://ia.javiergimenez.es:5500/transcribe
!curl -F "file=@grabacion.ogg"  http://localhost:5500/transcribe

In [12]:
!cd assets && wget http://localhost:5500/all_conversation

--2024-02-13 16:19:18--  http://localhost:5500/all_conversation
Resolving localhost (localhost)... 127.0.0.1
Connecting to localhost (localhost)|127.0.0.1|:5500... connected.
HTTP request sent, awaiting response... 404 NOT FOUND
2024-02-13 16:19:18 ERROR 404: NOT FOUND.



# Scripts varios

In [1]:
import os
import pandas as pd

def generar_csv_con_nombres_de_archivos(directorio):
    # Lista para almacenar los nombres de los archivos sin la extensión
    nombres_de_archivos = []
    
    # Recorre los archivos en el directorio
    for archivo in os.listdir(directorio):
        # Comprueba si el archivo es un .png
        if archivo.endswith('.png'):
            # Añade el nombre del archivo sin la extensión a la lista
            nombres_de_archivos.append(os.path.splitext(archivo)[0])
    
    # Crea un DataFrame de pandas con los nombres de los archivos
    df = pd.DataFrame(nombres_de_archivos, columns=['Personaje'])
    
    # Guarda el DataFrame en un archivo .csv
    df.to_csv('Personajes_ficcion.csv', index=False)
    
    print("Archivo 'Personajes_ficcion.csv' generado exitosamente.")

generar_csv_con_nombres_de_archivos('ImagenesjuegoConversacional')


Archivo 'Personajes_ficcion.csv' generado exitosamente.


In [20]:
from PIL import Image
import os

def reducir_imagenes(directorio, subcarpeta='reducidas'):
    # Crea la subcarpeta si no existe
    ruta_subcarpeta = os.path.join(directorio, subcarpeta)
    if not os.path.exists(ruta_subcarpeta):
        os.makedirs(ruta_subcarpeta)
    
    for archivo in os.listdir(directorio):
        if archivo.endswith('.png'):
            ruta_completa = os.path.join(directorio, archivo)
            with Image.open(ruta_completa) as img:
                # Redimensionar la imagen a 256x256
                img_reducida = img.resize((256, 256), Image.Resampling.LANCZOS)
                
                # Construye la ruta de la nueva imagen en la subcarpeta
                ruta_guardado = os.path.join(ruta_subcarpeta, archivo)
                
                # Guardar la imagen reducida en la subcarpeta
                img_reducida.save(ruta_guardado)

    print(f"Imágenes reducidas guardadas en '{ruta_subcarpeta}'.")

reducir_imagenes('ImagenesjuegoConversacional/historicos/')


Imágenes reducidas guardadas en 'ImagenesjuegoConversacional/historicos/reducidas'.


In [21]:
from PIL import Image
import os

def comprimir_imagenes_png(directorio, subcarpeta='comprimidas'):
    # Crea la subcarpeta si no existe
    ruta_subcarpeta = os.path.join(directorio, subcarpeta)
    if not os.path.exists(ruta_subcarpeta):
        os.makedirs(ruta_subcarpeta)
    
    for archivo in os.listdir(directorio):
        if archivo.endswith('.png'):
            ruta_completa = os.path.join(directorio, archivo)
            with Image.open(ruta_completa) as img:
                # La imagen ya tiene el tamaño deseado, así que sólo aplicamos compresión
                
                # Construye la ruta de la nueva imagen en la subcarpeta
                ruta_guardado = os.path.join(ruta_subcarpeta, archivo)
                
                # Guardar la imagen con compresión y optimización
                img.save(ruta_guardado, optimize=True, compress_level=9)

    print(f"Imágenes comprimidas guardadas en '{ruta_subcarpeta}'.")

comprimir_imagenes_png('ImagenesjuegoConversacional/ficcion/')


Imágenes comprimidas guardadas en 'ImagenesjuegoConversacional/ficcion/comprimidas'.


In [22]:
import subprocess
import os

def comprimir_imagenes_png_con_pngquant(directorio, subcarpeta='comprimidas'):
    ruta_subcarpeta = os.path.join(directorio, subcarpeta)
    if not os.path.exists(ruta_subcarpeta):
        os.makedirs(ruta_subcarpeta)

    for archivo in os.listdir(directorio):
        if archivo.endswith('.png'):
            ruta_completa = os.path.join(directorio, archivo)
            ruta_destino = os.path.join(ruta_subcarpeta, archivo)
            
            # Ejecutar pngquant para comprimir la imagen
            # --force sobrescribe archivos existentes
            # --skip-if-larger solo guarda la imagen comprimida si es más pequeña que la original
            comando = f'pngquant --force --skip-if-larger --output "{ruta_destino}" -- "{ruta_completa}"'
            subprocess.run(comando, shell=True)

    print(f"Imágenes comprimidas guardadas en '{ruta_subcarpeta}'.")

comprimir_imagenes_png_con_pngquant('ImagenesjuegoConversacional/ficcion/')


Imágenes comprimidas guardadas en 'ImagenesjuegoConversacional/ficcion/comprimidas'.


# Verificamos funcionamiento de generador de voz sintética.

In [90]:
import re
from num2words import num2words

def number_to_words(num_str):
    try:
        num = int(num_str)
        return num2words(num, lang='es')
    except ValueError:
        return "Por favor, introduzca un número válido."

def process_numbers_in_line(line):
    def replace_with_words(match):
        return number_to_words(match.group())
    
    return re.sub(r'\b\d+\b', replace_with_words, line)

# Ejemplo de uso
line = "Tengo 3 manzanas y 15 naranjas, sumando un total de 18 frutas."
new_line = process_numbers_in_line(line)
print(new_line)  
# Salida: "Tengo tres manzanas y quince naranjas, sumando un total de dieciocho frutas."

Tengo tres manzanas y quince naranjas, sumando un total de dieciocho frutas.


In [91]:
# Diccionario con las traducciones


def process_abrev(line):
    translations = {
    'Dr': 'doctor',
    'Sr': 'señor',
    'Sra': 'señora',
    # Añade más traducciones aquí
}
    for abbr, full in translations.items():
        line = line.replace(f'{abbr}.', full)
        line = line.replace(f'{abbr} ', f'{full} ')
    return line

# Ejemplo de uso
line1 = 'El Dr. está hablando con el Sr. Pérez y la Sra. Gómez.'
line2 = 'Buenos días Sr. Gómez.'

new_line1 = process_abrev(line1)
new_line2 = process_abrev(line2)

print(new_line1)  # Debería imprimir: 'El Doctor está hablando con el Señor Pérez y la Señora Gómez.'
print(new_line2)  # Debería imprimir: 'Buenos días Señor Gómez.'

El doctor está hablando con el señor Pérez y la señora Gómez.
Buenos días señor Gómez.


In [92]:
# Diccionario con las traducciones


def otras_traducciones(line):

    translations = {
    '-': ',',
    '—': ',', 
    '%': ' por ciento '
    # Añade más traducciones aquí
    }
    
    for old, new in translations.items():
        line = line.replace(old, new)
    return line

# Ejemplo de uso
line1 = 'Hola —cómo estás—?, bien al 100%, gracias.'
line2 = "El relato tuvo su origen en los primeros capítulos del Libro Rojo, compuesto por Bilbo Bolsón —el primer hobbit que fue famoso en el mundo entero— y que él tituló Historia de una ida y de una vuelta,"

new_line1 = otras_traducciones(line1)
new_line2 = otras_traducciones(line2)

print(new_line1)  # Debería imprimir: 'Hola, cómo estás?'
print(new_line2)  # Debería imprimir: 'Bien, gracias.'

Hola ,cómo estás,?, bien al 100 por ciento , gracias.
El relato tuvo su origen en los primeros capítulos del Libro Rojo, compuesto por Bilbo Bolsón ,el primer hobbit que fue famoso en el mundo entero, y que él tituló Historia de una ida y de una vuelta,


In [93]:
def preprocesado_al_modelo(line):
    line_with_numbers = process_numbers_in_line(line)
    line_with_both = process_abrev(line_with_numbers)
    line_with_all = otras_traducciones(line_with_both)
    return line_with_all

In [94]:
import os
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'
print(os.environ['TF_ENABLE_ONEDNN_OPTS'])  # Salida: mi_valor

0


In [95]:
from fairseq.checkpoint_utils import load_model_ensemble_and_task_from_hf_hub
from fairseq.models.text_to_speech.hub_interface import TTSHubInterface
import IPython.display as ipd
import torch  # Importamos PyTorch para poder usar la función `to()`

# Función para mover recursivamente todos los tensores en una estructura anidada a un dispositivo
def move_to_device(obj, device):
    if isinstance(obj, torch.Tensor):
        return obj.to(device)
    elif isinstance(obj, dict):
        return {key: move_to_device(value, device) for key, value in obj.items()}
    elif isinstance(obj, list):
        return [move_to_device(item, device) for item in obj]
    else:
        return obj
    

# Cargamos el modelo y la configuración desde el modelo preentrenado de Hugging Face
models, cfg, task = load_model_ensemble_and_task_from_hf_hub(
    "facebook/tts_transformer-es-css10",
    arg_overrides={"vocoder": "hifigan", "fp16": False}
)
model = models[0]

# Movemos el modelo al dispositivo GPU
model = model.to('cuda:0')

# Actualizamos la configuración con los datos del task
TTSHubInterface.update_cfg_with_data_cfg(cfg, task.data_cfg)

# Creamos el generador
generator = task.build_generator([model], cfg)


# Texto a convertir en voz
text = "Hola, esta es una prueba. Quiero saber hasta dónde llega el modelo."

# Preparamos los datos de entrada para el modelo
sample = TTSHubInterface.get_model_input(task, text)

# Movemos los datos al dispositivo GPU
sample = move_to_device(sample, 'cuda:0')

# Realizamos la predicción
wav, rate = TTSHubInterface.get_prediction(task, model, generator, sample)

# Convertimos el resultado al formato de audio y lo reproducimos
ipd.Audio(wav.cpu(), rate=rate)  # Usamos `.cpu()` para mover los datos de vuelta a la CPU antes de reproducir el audio


2024-02-01 15:46:45 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX


Fetching 10 files:   0%|          | 0/10 [00:00<?, ?it/s]

2024-02-01 15:46:46 | INFO | fairseq.tasks.speech_to_text | dictionary size (spm_char.txt): 107
/home/javier/miniconda3/envs/mistral/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")
2024-02-01 15:46:46 | INFO | fairseq.models.text_to_speech.vocoder | loaded HiFiGAN checkpoint from /home/javier/.cache/fairseq/models--facebook--tts_transformer-es-css10/snapshots/f52cf36f741df546bed60cdd5e6b71e0b85378c1/hifigan.bin
2024-02-01 15:46:46 | INFO | fairseq.models.text_to_speech.vocoder | loaded HiFiGAN checkpoint from /home/javier/.cache/fairseq/models--facebook--tts_transformer-es-css10/snapshots/f52cf36f741df546bed60cdd5e6b71e0b85378c1/hifigan.bin
/home/javier/miniconda3/envs/mistral/lib/python3.10/site-packages/torch/nn/modules/conv.py:306: UserWarning:

In [7]:
import re

def dividir_texto_con_minimo_palabras(texto, min_palabras=8):
    partes = re.split(r'([.;:?!])', texto)
    partes_filtradas = [parte.strip() for parte in partes if parte.strip()]
    partes_combinadas = []
    parte_actual = ''

    for parte in partes_filtradas:
        if parte in '.;:?!':
            parte_actual += parte
            if len(parte_actual.split()) >= min_palabras:
                partes_combinadas.append(parte_actual)
                parte_actual = ''
            else:
                parte_actual += ' '
        else:
            parte_actual += parte + ' '

    if len(parte_actual.strip()) > 10:
        partes_combinadas.append(parte_actual.strip())

    return partes_combinadas

texto = "Genial, gracias por venir hoy. Como sabes, estamos buscando a alguien con fuertes habilidades de programación, particularmente en Python y Java. ¿Puedes contarme sobre tu experiencia en estos lenguajes?"
texto = "Muchas gracias por venir a nuestra empresa, Javier. Como sabes, MAgIc se centra en encontrar soluciones de Inteligencia Artificial (IA) para empresas mediante el análisis y manipulación de grandes cantidades de datos, incluyendo matemáticas avanzadas, ciencia de datos e IA en particular, junto con la capacidad de generar resultados financieramente favorables."
lista_dividida = dividir_texto_con_minimo_palabras(texto)

print(lista_dividida)


['Muchas gracias por venir a nuestra empresa, Javier .', 'Como sabes, MAgIc se centra en encontrar soluciones de Inteligencia Artificial (IA) para empresas mediante el análisis y manipulación de grandes cantidades de datos, incluyendo matemáticas avanzadas, ciencia de datos e IA en particular, junto con la capacidad de generar resultados financieramente favorables .']


## Prueba con traducción larga

In [97]:
# Texto a convertir en voz
text = "Genial, gracias por venir hoy. Como sabes, estamos buscando a alguien con fuertes habilidades de programación, particularmente en Python y Java. ¿Puedes contarme sobre tu experiencia en estos lenguajes?"

texto = "Es bueno verte, Javier. Como CEO de mAgIc, estamos comprometidos con el desarrollo de soluciones de IA de vanguardia para nuestros clientes. ¿Puedes contarnos sobre tu experiencia con lenguajes de programación como Python y R? ¿Qué tan cómodo estás con la ciencia de datos y algoritmos de aprendizaje automático como Gradient Boosting Machines y XGBoost?\""

lista_textos = dividir_texto_con_minimo_palabras(text, min_palabras=7)

for texto in lista_textos:
    # Preparamos los datos de entrada para el modelo
    sample = TTSHubInterface.get_model_input(task, texto)

    # Movemos los datos al dispositivo GPU
    sample = move_to_device(sample, 'cuda:0')

    # Realizamos la predicción
    wav, rate = TTSHubInterface.get_prediction(task, model, generator, sample)

    # Convertimos el resultado al formato de audio y lo reproducimos
    display(ipd.Audio(wav.cpu(), rate=rate))  # Usamos `.cpu()` para mover los datos de vuelta a la CPU antes de reproducir el audio


In [100]:
import torchaudio

import re

def dividir_texto_con_minimo_palabras(texto, min_palabras=8):
    partes = re.split(r'([.;:?!])', texto)
    partes_filtradas = [parte.strip() for parte in partes if parte.strip()]
    partes_combinadas = []
    parte_actual = ''

    for parte in partes_filtradas:
        if parte in '.;:?!':
            parte_actual += parte
            if len(parte_actual.split()) >= min_palabras:
                partes_combinadas.append(parte_actual)
                parte_actual = ''
            else:
                parte_actual += ' '
        else:
            parte_actual += parte + ' '

    if parte_actual.strip():
        partes_combinadas.append(parte_actual.strip())

    return partes_combinadas

def combinar_audios(audios_temporales):
    # Cargar el primer archivo de audio para inicializar la concatenación
    wav_total, rate = torchaudio.load(audios_temporales[0])

    # Iterar sobre los archivos restantes y concatenarlos
    for archivo in audios_temporales[1:]:
        wav, _ = torchaudio.load(archivo)
        wav_total = torch.cat((wav_total, wav), 1)

    # Guardar el audio combinado en un archivo final
    torchaudio.save('temporal_combinado.wav', wav_total, rate)

    return 'temporal_combinado.wav'

def voz_sintetica_spanish(text):
    text = preprocesado_al_modelo(text)

    lista_dividida = dividir_texto_con_minimo_palabras(text)

    audios_temporales = []
    
    for parte in lista_dividida:
        # Preparamos los datos de entrada para el modelo
        sample = TTSHubInterface.get_model_input(task, parte)

        # Movemos los datos al dispositivo GPU
        sample = move_to_device(sample, 'cuda:0')

        # Realizamos la predicción
        wav, rate = TTSHubInterface.get_prediction(task, model, generator, sample)

        if len(wav.shape) == 1:
            wav = wav.unsqueeze(0)
            
        # temp_file_name = "Temporal.wav"
        temp_file_name = f"temporal_{parte[:10]}.wav"
        torchaudio.save(temp_file_name, wav.to('cpu'), rate)
        audios_temporales.append(temp_file_name)

        archivo_final = combinar_audios(audios_temporales)

text = "Genial, gracias por venir hoy. Como sabes, estamos buscando a alguien con fuertes habilidades de programación, matemáticas, ciencia de datos, inteligencia artificial y experiencia en PNL para unirse a nuestro equipo. Comencemos con los lenguajes de programación, ¿cuáles eres más competente?"

voz_sintetica_spanish(text)

QUitar silencios

In [108]:
from pydub import AudioSegment, silence

def quitar_silencios(input_filepath, output_filepath, min_silence_len=1500, new_silence_len=750, silence_thresh=-60):
    """
    Elimina silencios largos de un archivo de audio.

    Parámetros:
    - input_filepath: ruta al archivo de audio de entrada (MP3).
    - output_filepath: ruta al archivo de audio de salida (MP3).
    - min_silence_len: duración mínima del silencio a eliminar (en milisegundos).
    - new_silence_len: duración de los nuevos segmentos de silencio (en milisegundos).
    - silence_thresh: umbral de silencio (en dB).
    """
    
    # Cargar el archivo de audio
    audio_segment = AudioSegment.from_wav(input_filepath)
    
    # Encuentra los segmentos de audio separados por silencios
    segments = silence.split_on_silence(audio_segment, min_silence_len=min_silence_len, silence_thresh=silence_thresh)

    # Crear un nuevo segmento de audio con silencios ajustados
    new_audio_segment = AudioSegment.empty()
    silence_chunk = AudioSegment.silent(duration=new_silence_len)  # Chunk de silencio de la nueva duración

    # Añade cada segmento de audio al nuevo audio, intercalando con los nuevos segmentos de silencio
    for segment in segments:
        new_audio_segment += segment + silence_chunk

    # Removemos el último chunk de silencio añadido
    new_audio_segment = new_audio_segment[:-new_silence_len]

    # Guarda el nuevo archivo de audio
    new_audio_segment.export(output_filepath, format="wav")


In [109]:
def dividir_texto_con_minimo_palabras(texto, min_palabras=8):
    partes = re.split(r'([.,;:?!])', texto)
    partes_filtradas = [parte.strip() for parte in partes if parte.strip()]
    partes_combinadas = []
    parte_actual = ''

    for parte in partes_filtradas:
        if parte in '.,;:?!':
            parte_actual += parte
            if len(parte_actual.split()) >= min_palabras:
                partes_combinadas.append(parte_actual)
                parte_actual = ''
            else:
                parte_actual += ' '
        else:
            parte_actual += parte + ' '

    if parte_actual.strip():
        partes_combinadas.append(parte_actual.strip())

    return partes_combinadas

def combinar_audios(audios_temporales):
    audio_combinado = "audio_combinado.wav"
    # Cargar el primer archivo de audio para inicializar la concatenación
    wav_total, rate = torchaudio.load(audios_temporales[0])

    # Iterar sobre los archivos restantes y concatenarlos
    for archivo in audios_temporales[1:]:
        wav, _ = torchaudio.load(archivo)
        wav_total = torch.cat((wav_total, wav), 1)

    # Guardar el audio combinado en un archivo final
    torchaudio.save(audio_combinado, wav_total, rate)

    return audio_combinado

def voz_sintetica_spanish(text):
    text = preprocesado_al_modelo(text)

    lista_dividida = dividir_texto_con_minimo_palabras(text)

    audios_temporales = []
    
    for parte in lista_dividida:
        # Preparamos los datos de entrada para el modelo
        sample = TTSHubInterface.get_model_input(task, parte)

        # Movemos los datos al dispositivo GPU
        sample = move_to_device(sample, 'cuda:0')

        # Realizamos la predicción
        wav, rate = TTSHubInterface.get_prediction(task, model, generator, sample)

        if len(wav.shape) == 1:
            wav = wav.unsqueeze(0)
            
        # temp_file_name = "Temporal.wav"
        temp_file_name = f"temporal_{parte[:10]}.wav"
        sin_silencios = "quitar_silencios.wav"
        torchaudio.save(temp_file_name, wav.to('cpu'), rate)
        audios_temporales.append(temp_file_name)

        combinado = combinar_audios(audios_temporales)
        quitar_silencios(combinado, sin_silencios, min_silence_len=1500, new_silence_len=750, silence_thresh=-60)

text = "Genial, gracias por venir hoy. Como sabes, estamos buscando a alguien con fuertes habilidades de programación, particularmente en Python y Java. ¿Puedes contarme sobre tu experiencia en estos lenguajes?"

voz_sintetica_spanish(text)


In [ ]:
def spanish_t2s(text):
    text = preprocesado_al_modelo(text)
    # Preparamos los datos de entrada para el modelo
    sample = TTSHubInterface.get_model_input(task, text)

    # Movemos los datos al dispositivo GPU
    sample = move_to_device(sample, 'cuda:0')

    # Realizamos la predicción
    wav, rate = TTSHubInterface.get_prediction(task, model, generator, sample)

    return wav, rate

# pruebas varias

In [8]:
%%html

<div id="audioPlayerContainer"></div>
<button id="recordButton" style="width: 100%; height: 50px;">Pulsa para grabar/detener</button>

<!-- Estilos para textarea y botón de envío -->
<div style="margin-top: 10px;">
    <textarea id="textInput" placeholder="Escribe tu texto aquí" rows="4" style="width: calc(100% - 90px); display: inline-block;"></textarea>
    <button id="sendTextButton" style="width: 80px; height: 70px; vertical-align: top;">Enviar Texto</button>
</div>


<div id="responseText" style="margin-top: 20px; border: 1px solid #ddd; padding: 10px;"></div>
<script>
let recordButton = document.getElementById("recordButton");
let chunks = [];
let mediaRecorder;
let isRecording = false;

navigator.mediaDevices.getUserMedia({ audio: true })
.then(stream => {
    mediaRecorder = new MediaRecorder(stream);
    mediaRecorder.ondataavailable = event => {
        chunks.push(event.data);
    };
    mediaRecorder.onstop = () => {
        console.log('mediaRecorder Detenido!!');
        let blob = new Blob(chunks, { 'type': 'audio/ogg; codecs=opus' });
        enviarAudioAlServidor(blob);
        chunks = [];
    };
});

recordButton.onclick = () => {
    if (!isRecording) {
        mediaRecorder.start();
        isRecording = true;
        recordButton.textContent = 'Grabando...';
    } else {
        mediaRecorder.stop();
        isRecording = false;
        recordButton.textContent = 'Pulsa para grabar/detener';
    }
};

function enviarAudioAlServidor(blob) {
    let formData = new FormData();
    formData.append('file', blob, 'grabacion.ogg');

    fetch('http://localhost:5500/transcribe', {
    // fetch('http://ia.javiergimenez.es:5500/transcribe', {

        method: 'POST',
        body: formData,
    })
    .then(response => response.json())
    .then(data => {
        updateResponseText("yo: " + data.entrada + "\n\n" + (data.entrada_traducida ? "yo (traducción): " + data.entrada_traducida + "\n" : "\n") + "\n***********************************\n" + "respuesta: " + data.respuesta + "\n\n" + (data.respuesta_traducida ? "Respuesta (traducida): " + data.respuesta_traducida + "\n" : ""));
        if (data.respuesta_traducida) {
            obtenerYReproducirAudio(data.respuesta_traducida);
        }
        else if (data.respuesta) {
            obtenerYReproducirAudio(data.respuesta);
        }
    })
    .catch(error => {
        console.error('Error al enviar el audio:', error);
    });
}

// Funcionalidad para enviar texto al servidor y limpiar el textarea
document.getElementById("sendTextButton").onclick = () => {
    let textInput = document.getElementById("textInput");
    let texto = textInput.value;
    if (texto) {
        enviarTextoAlServidor(texto);
        textInput.value = ''; // Limpiar el textarea después de enviar
    }
};

function enviarTextoAlServidor(texto) {
    fetch('http://localhost:5500/texto', {
    // fetch('http://ia.javiergimenez.es:5500/texto', {

        method: 'POST',
        headers: {
            'Content-Type': 'application/json'
        },
        body: JSON.stringify({ texto: texto })
    })
    .then(response => response.json())
    .then(data => {
        updateResponseText("yo: " + data.entrada + "\n\n" + (data.entrada_traducida ? "yo (traducción): " + data.entrada_traducida + "\n" : "\n") + "\n***********************************\n" + "respuesta: " + data.respuesta + "\n\n" + (data.respuesta_traducida ? "Respuesta (traducida): " + data.respuesta_traducida + "\n" : ""));
        if (data.respuesta_traducida) {
            obtenerYReproducirAudio(data.respuesta_traducida);
        }
        else if (data.respuesta) {
            obtenerYReproducirAudio(data.respuesta);
        }
    })
    .catch(error => {
        console.error('Error al enviar el texto:', error);
    });
}

function updateResponseText(text) {
    document.getElementById('responseText').innerText = text;
}


function obtenerYReproducirAll() {
    fetch('http://localhost:5500/all_conversation', {
        method: 'GET',
    })
    .then(response => response.json())
    .then(data => {
        if (data.audio_base64) {
            createAudioAllPlayer(data.audio_base64);
        }
    })
    .catch(error => {
        console.error('Error al obtener el audio:', error);
    });
}


function createAudioAllPlayer(base64Audio) {
    let audioContainer = document.getElementById('audioPlayerAllContainer');
    let audioSrc = `data:audio/wav;base64,${base64Audio}`;
    let audioPlayer = document.createElement('audio');
    audioPlayer.src = audioSrc;
    audioPlayer.controls = true;
    audioPlayer.autoplay = true;
    audioContainer.innerHTML = '';
    audioContainer.appendChild(audioPlayer);
}

function obtenerYReproducirAudio(texto) {
    fetch('http://localhost:5500/audio', {

        method: 'POST',
        headers: {
            'Content-Type': 'application/json'
        },
        body: JSON.stringify({ texto: texto })
    })
    .then(response => response.json())
    .then(data => {
        if (data.audio_base64) {
            createAudioPlayer(data.audio_base64);
        }
    })
    .catch(error => {
        console.error('Error al obtener el audio:', error);
    });
}

function createAudioPlayer(base64Audio) {
    let audioContainer = document.getElementById('audioPlayerContainer');
    let audioSrc = `data:audio/wav;base64,${base64Audio}`;
    let audioPlayer = document.createElement('audio');
    audioPlayer.src = audioSrc;
    audioPlayer.controls = true;
    audioPlayer.autoplay = true;
    audioContainer.innerHTML = '';
    audioContainer.appendChild(audioPlayer);
}
</script>


In [ ]:
!ls -l received_audio_1707815744778.ogg
!ffmpeg -y -i received_audio_1707816813317.ogg output.wav
def convert_ogg_to_mp3(source_ogg_path, target_mp3_path):
    """
    Utiliza ffmpeg para convertir un archivo .ogg a .mp3.
    """
    command = ['ffmpeg', '-y', '-i', source_ogg_path, target_mp3_path]
    process = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

    # Si el comando falla, imprime la salida de error
    if process.returncode != 0:
        print(f"Error al convertir {source_ogg_path} a {target_mp3_path}")
        print("Salida de error de ffmpeg:")
        print(process.stderr.decode())

import threading

def add_audio_to_conversation_async(source_path):
    """
    Convierte el archivo .ogg a .mp3 y lo añade al archivo 'conversacion.mp3' en segundo plano.
    """
    def task():
        # Convertir .ogg a .mp3
        temp_mp3_path = source_path.replace('.ogg', '.mp3')
        convert_ogg_to_mp3(source_path, temp_mp3_path)

        # Cargar el archivo de audio .mp3 convertido
        sound = AudioSegment.from_mp3(temp_mp3_path)
        
        # Nombre del archivo de conversación
        conversation_file = 'conversacion.mp3'
        
        # Cargar o inicializar el archivo de conversación
        if os.path.exists(conversation_file):
            conversation_audio = AudioSegment.from_mp3(conversation_file)
            combined_audio = conversation_audio + sound
        else:
            combined_audio = sound
        
        # Guardar el audio combinado en el archivo de conversación
        combined_audio.export(conversation_file, format='mp3')
        
        # Eliminar los archivos temporales
        os.remove(source_path)
        os.remove(temp_mp3_path)

    # Iniciar la tarea en un hilo separado
    thread = threading.Thread(target=task)
    thread.start()        

source_ogg_path = "received_audio_1707816813317.ogg"
add_audio_to_conversation_async(source_ogg_path)
from pydub import AudioSegment 
sound = AudioSegment.from_ogg("./received_audio_1707816813317.ogg")
print(len(sound))

In [ ]:
import subprocess
from pydub import AudioSegment
import os

def convert_ogg_to_mp3(source_ogg_path, target_mp3_path):
    """
    Utiliza ffmpeg para convertir un archivo .ogg a .mp3.
    """
    command = ['ffmpeg', '-y', '-i', source_ogg_path, target_mp3_path]
    subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

def add_audio_to_conversation(source_path, conversation_path='conversacion.mp3'):
    """
    Convierte el archivo .ogg a .mp3 y lo añade al archivo de conversación.
    """
    # Nombre temporal para el archivo convertido
    temp_mp3_path = source_path.replace('.ogg', '.mp3')

    # Convertir .ogg a .mp3
    convert_ogg_to_mp3(source_path, temp_mp3_path)

    # Cargar el archivo .mp3 convertido
    new_audio = AudioSegment.from_mp3(temp_mp3_path)

    # Cargar o inicializar el archivo de conversación
    if os.path.exists(conversation_path):
        conversation_audio = AudioSegment.from_mp3(conversation_path)
        combined_audio = conversation_audio + new_audio
    else:
        combined_audio = new_audio

    # Guardar el audio combinado
    combined_audio.export(conversation_path, format='mp3')

    # Limpiar: eliminar los archivos temporales
    os.remove(temp_mp3_path)
    os.remove(source_path)

# Ejemplo de uso:
source_ogg_path = "received_audio_1707816813317.ogg"
add_audio_to_conversation(source_ogg_path)
